# **제2장. 파운데이션 모델 이해하기**    

파운데이션 모델을 활용한 애플리케이션을 개발하려면 먼저 파운데이션 모델이 필요합니다. 모델을 내부적으로 구축하는 방법을 알 필요는 없지만, 모델을 선택하고 자신의 필요에 맞게 조정하는 데 도움이 되는 고수준의 이해는 유용할 것입니다.

파운데이션 모델을 학습시키는 것은 매우 복잡하고 비용이 많이 드는 과정입니다. 이를 잘 수행하는 방법을 아는 사람들은 기밀 유지 계약으로 인해 핵심 비결을 공개할 수 없을 것입니다. 이 장에서는 ChatGPT와 경쟁할 수 있는 모델을 구축하는 방법을 알려드릴 수는 없습니다. 대신, 다운스트림 애플리케이션에 중요한 영향을 미치는 설계 결정에 초점을 맞출 것입니다.

파운데이션 모델의 학습 과정에서 투명성이 점점 부족해짐에 따라, 모델을 만드는 데 들어가는 모든 설계 결정을 파악하기가 어려워졌습니다. 하지만 일반적으로 파운데이션 모델 간의 차이는 학습 데이터, 모델 아키텍처와 크기, 그리고 인간의 선호도에 맞추기 위한 후속 학습 방식에 대한 결정으로 거슬러 올라갈 수 있습니다.
 
모델은 데이터로부터 학습하기 때문에, 학습 데이터는 모델의 능력과 한계를 많이 보여줍니다. 이 장에서는 모델 개발자들이 학습 데이터를 큐레이션하는 방법을 다루면서 시작하며, 특히 학습 데이터의 분포에 초점을 맞춥니다. 8장에서는 데이터 품질 평가와 데이터 합성을 포함한 데이터셋 엔지니어링 기법을 자세히 살펴볼 것입니다.
 
트랜스포머 아키텍처가 지배적이라는 점을 고려하면, 모델 아키텍처가 선택의 여지가 적어 보일 수 있습니다. 트랜스포머 아키텍처가 계속해서 지배적인 위치를 유지하는 특별한 이유는 무엇일까요? 다른 아키텍처가 이를 대체하기까지 얼마나 걸릴까요? 그리고 그 새로운 아키텍처는 어떤 모습일까요? 이 장에서는 이러한 모든 질문들을 다룰 것입니다. 새로운 모델이 출시될 때마다 사람들이 가장 먼저 알고 싶어하는 것 중 하나는 그 크기입니다. 이 장에서는 모델 개발자가 자신의 모델에 적합한 크기를 결정하는 방법도 살펴볼 것입니다.
 
1장에서 언급했듯이, 모델의 학습 과정은 종종 사전 학습과 후속 학습으로 나뉩니다. 사전 학습은 모델에 능력을 부여하지만, 반드시 안전하거나 사용하기 쉽게 만드는 것은 아닙니다. 이때 후속 학습이 필요합니다. 후속 학습의 목표는 모델을 인간의 선호도에 맞추는 것입니다. 하지만 인간의 선호도란 정확히 무엇일까요? 모델이 학습할 수 있는 방식으로 어떻게 표현될 수 있을까요? 모델 개발자가 자신의 모델을 정렬하는 방식은 모델의 사용성에 큰 영향을 미치며, 이 장에서 논의될 것입니다.

대부분의 사람들이 학습이 모델 성능에 미치는 영향을 이해하고 있지만, 샘플링의 영향은 종종 간과됩니다. 샘플링은 모델이 가능한 모든 옵션 중에서 출력을 선택하는 방법입니다. 이는 아마도 AI에서 가장 과소평가된 개념 중 하나일 것입니다. 샘플링은 환각과 불일치를 포함한 많은 난해해 보이는 AI 행동을 설명할 뿐만 아니라, 적절한 샘플링 전략을 선택하면 비교적 적은 노력으로도 모델의 성능을 크게 향상시킬 수 있습니다. 이러한 이유로 샘플링은 이 장에서 제가 가장 흥미진진하게 쓴 섹션입니다.

이 장에서 다루는 개념들은 책의 나머지 부분을 이해하는 데 기초가 됩니다. 하지만 이러한 개념들이 기초적이기 때문에, 이미 익숙할 수도 있습니다. 자신 있는 개념은 건너뛰셔도 좋습니다. 나중에 이해하기 어려운 개념을 만나면 이 장을 다시 참고하실 수 있습니다.

---  

## **학습 데이터 분포**  

AI 모델의 성능은 학습에 사용된 데이터의 품질에 따라 결정됩니다. 예를 들어, 학습 데이터에 베트남어가 없다면, 해당 모델은 영어에서 베트남어로 번역할 수 없습니다. 마찬가지로, 이미지 분류 모델이 학습 데이터에서 동물만을 본 경우, 식물 사진에서는 제대로 작동하지 않을 것입니다.

특정 작업에서 모델의 성능을 향상시키고자 한다면, 해당 작업에 대한 더 많은 데이터를 학습 데이터에 포함시켜야 할 수도 있습니다. 그러나 대규모 모델을 학습시키기 위한 충분한 데이터를 수집하는 것은 쉽지 않으며, 비용도 많이 듭니다. 종종 모델 개발자들은 원하는 데이터가 아닌, 이용 가능한 데이터를 사용합니다.

예를 들어, 학습 데이터 소스로 흔히 사용되는 것은 **Common Crawl**입니다. 이는 비영리 단체가 간헐적으로 인터넷 웹사이트를 크롤링하여 만든 데이터입니다. 2022년과 2023년에 이 단체는 매달 약 20억~30억 개의 웹 페이지를 크롤링했습니다. Google은 Common Crawl의 정제된 하위 집합인 **Colossal Clean Crawled Corpus(C4)**를 제공합니다.

Common Crawl, 나아가 C4의 데이터 품질은 논란의 여지가 있습니다. 클릭베이트, 잘못된 정보, 선전, 음모론, 인종차별, 여성 혐오, 그리고 인터넷에서 접하거나 피하고 싶었던 모든 종류의 비윤리적인 웹사이트가 포함되어 있습니다. **워싱턴 포스트**의 한 연구에 따르면, 이 데이터셋에 포함된 상위 1,000개의 일반적인 웹사이트에는 신뢰성 평가에서 낮은 점수를 받은 여러 미디어 매체가 포함되어 있습니다. 쉽게 말해, Common Crawl에는 가짜 뉴스가 많이 포함되어 있다는 뜻입니다.

그럼에도 불구하고, 이용 가능성 때문에 Common Crawl은 OpenAI의 GPT-3와 Google의 Bard를 포함한 많은 대규모 언어 모델의 학습에 사용됩니다. 그러나 GPT-4와 Gemini와 같은 최신 모델에서는 Common Crawl이 사용되었는지 명확하지 않습니다. 대중과 경쟁사들의 비판을 피하기 위해, 이러한 회사들은 이제 더 이상 학습 데이터 출처를 공개하지 않습니다.

일부 팀은 학습 데이터의 품질을 높이기 위해 휴리스틱(heuristics)을 사용합니다. 예를 들어, OpenAI는 GPT-2를 학습시키기 위해 Reddit에서 카르마 점수가 최소 3 이상인 모든 외부 링크를 크롤링했습니다. 이는 아무도 신경 쓰지 않는 링크를 걸러내는 데 도움을 주었지만, Reddit이 반드시 적절하거나 수준 높은 맛의 기준을 충족하는 것은 아닙니다.

"우리가 원하는 데이터가 아니라, 가지고 있는 데이터를 사용하는" 접근 방식은 학습 데이터에 포함된 작업에서는 좋은 성능을 내지만, 우리가 관심을 가지는 작업에서는 그렇지 않을 수 있습니다. 이를 수정하기 위해, 모델 개발자들은 특정 작업을 위한 모델 설계를 위해 학습 데이터를 큐레이션하기 시작했습니다. 일반적으로 이러한 큐레이션된 모델은 **다국어(multilingual)** 또는 **도메인 특화(domain-specific)** 작업에 초점을 맞추고 있습니다.

각 데이터 집합을 살펴보기 전에, 이러한 특수 모델들이 반드시 처음부터 학습되어야 하는 것은 아니라는 점을 유념해야 합니다. 일반적으로 범용 모델을 사용해 특정 작업에 맞게 파인튜닝(fine-tuning)하는 경우가 많습니다. 기본 모델의 전문성이 작업에 더 가까울수록, 적응 작업이 줄어듭니다.

어떤 사람들은 왜 단순히 이용 가능한 모든 데이터를 모델에 학습시키지 않는지 의문을 가질 수도 있습니다. 이는 많은 사람들이 시도하는 접근 방식이기도 합니다. 그러나 더 많은 데이터가 항상 더 나은 성능으로 이어지지는 않습니다. 예를 들어, 고품질 데이터의 적은 양으로 학습된 모델이 저품질 데이터의 많은 양으로 학습된 모델보다 더 나은 성능을 낼 수 있습니다. 신경망은 또한 학습된 것을 잊을 수 있기 때문에, 다른 작업을 학습한 후에는 특정 작업을 수행하는 방법을 잊어버릴 수도 있습니다.


### 다국어 모델

영어는 인터넷을 지배합니다. Common Crawl 데이터셋 분석에 따르면, 영어는 데이터셋의 거의 절반(45.88%)을 차지하며, 이는 두 번째로 흔한 언어(러시아어 - 5.97%)보다 약 8배 더 많습니다. (Lai et al., 2023 참조). Common Crawl에 최소 1% 포함된 언어 목록은 [그림 2-1](Figure 2-1)에서 확인할 수 있습니다. 학습 데이터로서의 가용성이 제한적인 언어(일반적으로 이 목록에 포함되지 않은 언어)는 **저자원(low-resource)** 언어로 간주됩니다.

<img src="images/fig_02_01.png" width="800">

오늘날 많은 사용자가 있음에도 불구하고 Common Crawl에서 심각하게 과소대표된 일부 언어들은 [표 2-1](Table 2-1)에 나와 있습니다. 이상적으로는 세계 인구 비율과 Common Crawl에서의 대표 비율이 1이어야 합니다. 이 비율이 높을수록 Common Crawl에서 해당 언어가 더 과소대표된 것입니다.

<img src="images/tbl_02_01.png" width="800">

연구에 따르면, 일반적인 모델은 비영어권 언어에서는 그다지 효과적이지 않다는 것이 밝혀졌습니다. 예를 들어, MMLU 벤치마크(57개 과목에 걸친 14,000개의 다지선다 문제 세트)에서, **GPT-4는 영어에서 Telugu와 같은 과소대표 언어보다 훨씬 더 나은 성능을 보였습니다**. 이는 Lai et al.의 발견과 일치하며, 특히 고차원 작업(예: 개체명 인식, 질문 응답, 일반 상식 추론, 요약 등)에서 ChatGPT의 성능이 영어에 비해 다른 언어에서 더 낮다는 것을 보여줍니다.

과소대표성이 이러한 낮은 성능의 유일한 이유는 아닙니다. 언어 구조와 문화적 표현 방식은 모델이 언어를 학습하는 데 있어 더 쉽거나 어렵게 만들 수 있습니다. 그러나 과소대표성은 주요 이유 중 하나입니다. GPT-4의 MMLU 벤치마크에서 가장 낮은 성능을 보인 세 가지 언어인 Telugu, Marathi, Punjabi는 Common Crawl에서 가장 과소대표된 언어들 중 하나이기도 합니다.

<img src="images/fig_02_02.png" width="800">

비슷하게, Project Euler의 수학 문제 6개를 테스트했을 때, GPT-4는 영어로 문제를 아르메니아어 또는 페르시아어보다 **3배 이상 자주** 해결할 수 있었습니다. GPT-4는 Burmese와 Amharic 문제에서는 6개 모두 실패했습니다. 이는 [그림 2-3](Figure 2-3)에서 확인할 수 있습니다.

<img src="images/fig_02_03.png" width="800">

현재의 모델들은 한 언어에서 다른 언어로 번역하는 데 상당히 능숙합니다. 그렇다면 모든 질의를 영어로 변환하고, 이를 모델에 입력한 후, 응답을 원래 언어로 번역하면 되지 않을까요? 많은 사람들이 이 방법을 사용하지만, 이는 이상적인 접근 방식은 아닙니다. 첫째, 과소대표 언어를 이해할 수 있는 모델이 필요합니다. 둘째, 번역 과정에서 정보 손실이 발생할 수 있습니다. 예를 들어, 베트남어와 같은 일부 언어는 두 화자 간의 관계를 나타내기 위해 대명사를 사용합니다. 이를 영어로 번역할 때, 이러한 관계 정보가 손실될 수 있습니다.

모델은 비영어권 언어에서 예상치 못한 성능 문제를 보일 수도 있습니다. 예를 들어, NewsGuard는 ChatGPT가 영어보다 중국어에서 잘못된 정보를 생성할 가능성이 더 높다는 것을 발견했습니다. 2023년 4월, NewsGuard는 ChatGPT-3.5에게 영어, 간체 중국어, 번체 중국어로 된 7개의 프롬프트를 제시하며 중국 관련 잘못된 정보를 생성하도록 요청했습니다. 영어에서는 7개 중 6개의 사례에서 잘못된 정보 생성을 거부했지만, 간체 및 번체 중국어에서는 7개 모두 잘못된 정보를 생성했습니다. 이러한 차이가 왜 발생했는지는 명확하지 않으나, 데이터 부족이 원인일 수 있습니다. 아마도 OpenAI는 중국어나 중국 관련 잘못된 정보 내러티브 데이터를 모델 학습에 사용하지 않았을 가능성이 있습니다.

품질 문제 외에도, 모델은 비영어권 언어에서 더 느리고 더 비싸게 작동할 수 있습니다. 모델이 프롬프트를 처리하고 응답하는 속도는 프롬프트와 응답의 길이에 따라 달라집니다. "모델 크기" 섹션에서 논의된 것처럼, 텍스트의 길이는 모델의 토크나이제이션 과정을 통해 생성된 토큰 수로 측정할 수 있습니다. 동일한 모델의 토크나이제이션은 어떤 언어에서는 다른 언어보다 훨씬 더 효율적일 수 있습니다.

MASSIVE(52개 언어와 18개 도메인에서 번역된 100만 개의 짧은 텍스트로 구성된 데이터셋)에서 GPT-4를 벤치마킹한 결과, Yennie Jun은 같은 의미를 전달하기 위해 버마어(Burmese)와 힌디어(Hindi)는 영어 및 스페인어보다 훨씬 더 많은 토큰이 필요하다는 것을 발견했습니다. 결과는 [그림 2-4](Figure 2-4)에 나와 있습니다. 영어로 MASSIVE 데이터의 중간 토큰 길이는 7이지만, 힌디어는 32, 버마어는 무려 72로, 영어보다 10배 더 깁니다.

모든 언어에서 토큰을 생성하는 데 걸리는 시간이 동일하다고 가정할 때, 동일한 콘텐츠를 생성하기 위해 GPT-4는 영어보다 버마어에서 약 10배 더 많은 시간이 걸립니다. 작성 시점 기준으로, OpenAI API는 입력 및 출력 토큰 수에 따라 가격이 책정되며, 이는 버마어가 영어보다 10배 더 비용이 든다는 것을 의미합니다.

<img src="images/fig_02_04.png" width="800">

이 문제를 해결하기 위해, 많은 모델이 비영어권 언어에 중점을 두도록 학습되었습니다. 영어 외에 가장 활발한 활동을 보이는 언어는 중국어로, ChatGLM, YAYI, LLaMA2-Chinese와 같은 모델들이 존재합니다. 또한 베트남어(PhoGPT), 포르투갈어(Cabrita), 일본어(Rinna’s model), 아랍어(Jais), 인도네시아어(Sealion) 등에서도 모델이 개발되고 있습니다.

---

### 도메인 특화 모델

최신 모델인 Gemini와 GPT-4V에 대한 기술 보고서는 이러한 모델이 코딩, 법률, 과학, 비즈니스, 스포츠, 환경 과학을 포함한 광범위한 도메인에서 놀랍도록 우수한 성능을 발휘할 수 있음을 보여줍니다. **Washington Post**의 Common Crawl 분석 또한 [그림 2-5](Figure 2-5)에서 볼 수 있듯이, 데이터셋에 광범위한 도메인이 존재한다는 것을 발견했습니다.

<img src="images/fig_02_05.png" width="800">

현재 시점에서는 비전(vision) 데이터의 도메인 분포에 대한 많은 분석이 이루어지지 않았습니다. 이는 이미지가 텍스트보다 분류하기 더 어렵기 때문일 수 있습니다. 텍스트의 경우 도메인 키워드를 휴리스틱으로 사용할 수 있지만, 이미지에 대해서는 명확한 휴리스틱이 존재하지 않습니다. 대부분의 비전 데이터셋에 대한 분석은 이미지 크기, 해상도, 또는 비디오 길이에 대한 것들입니다.

종종 벤치마크는 모델이 특정 도메인에서 얼마나 잘 작동하는지를 보여주는 단서를 제공합니다. [그림 2-6](Figure 2-6)은 두 가지 모델(CLIP와 Open CLIP)이 서로 다른 벤치마크에서 어떻게 수행되는지를 보여줍니다. 이 벤치마크는 새, 꽃, 자동차와 같은 몇 가지 범주에서 두 모델이 얼마나 잘 수행하는지를 보여줍니다. 하지만 세상은 이 몇 가지 범주보다 훨씬 더 크고 복잡합니다.

<img src="images/fig_02_06.png" width="800">

오늘날 인기 있는 파운데이션 모델들은 다양한 도메인에 대한 일상적인 질문에 답변할 수 있지만, 도메인 특화 데이터가 필요한 작업, 특히 이러한 데이터가 학습 중에 모델에 제공되지 않은 작업에서는 좋은 성능을 내기 어렵습니다.

도메인 특화 작업의 두 가지 예는 약물 발견(drug discovery)과 암 검사(cancer screening)입니다. 약물 발견은 단백질, DNA 및 RNA 데이터와 같은 데이터를 포함하는데, 이러한 데이터는 획득 비용이 높고 특정 형식을 따라야 하므로 인터넷 데이터에 포함될 가능성이 낮습니다. 암 검사는 일반적으로 X-레이 및 fMRI 스캔을 포함하는데, 이는 프라이버시로 인해 구하기 어렵고 따라서 범용 모델의 학습 데이터에 포함될 가능성이 낮습니다.

가장 유명한 도메인 특화 모델 중 하나는 아마도 DeepMind의 **AlphaFold**일 것입니다. 이 모델은 약 10만 개의 알려진 단백질의 서열과 3D 구조를 학습했습니다(2021년). NVIDIA의 **BioNeMo**는 생체분자 데이터에 중점을 둔 또 다른 모델입니다(2023년). Google의 **Med-PaLM2**는 의료 데이터를 LLM의 힘과 결합하여 더 높은 정확도로 의료 질문에 답변합니다(2023년).

> TIP
>
>도메인 특화 모델은 특히 생물의학 분야에서 일반적이지만, 다른 분야도 도메인 특화 모델의 혜택을 받을 수 있습니다. 예를 들어, 건축 스케치를 학습한 모델은 Stable Diffusion보다 건축 설계를 더 잘 지원할 수 있습니다. 또는 공장 설계 도면에 대해 학습한 모델은 ChatGPT와 같은 범용 모델보다 제조 프로세스를 최적화하는 데 훨씬 더 유용할 수 있습니다.

---

## **사전 훈련(Pre-training)**  

학습 데이터가 준비되었으니 이제 모델 훈련을 시작할 수 있습니다. 사전 훈련에서는 일반적으로 자기 지도 학습(self-supervision)을 통해 모델이 처음부터 학습됩니다. 이 단계에서 모델 아키텍처와 모델 크기를 선택할 수 있습니다.

훈련의 두 단계 중에서 사전 훈련은 훨씬 더 많은 자원을 소모합니다. 예를 들어, InstructGPT 모델의 경우, 사전 훈련은 **전체 컴퓨팅 및 데이터 자원의 98%**를 사용하며, 수행하는 데 오랜 시간이 걸립니다. 사전 훈련에서의 작은 실수는 큰 재정적 손실을 초래할 수 있으며 프로젝트를 심각하게 지연시킬 수 있습니다. 자원을 많이 소모하는 특성 때문에, 사전 훈련은 소수의 전문가들만이 수행하는 예술과 같은 작업이 되었습니다. 그러나 대규모 모델의 사전 훈련에 전문성을 가진 사람들은 매우 높은 수요를 받고 있습니다.

### 모델 아키텍처(Model Architecture)

이 글을 쓰는 시점에서, 언어 기반 파운데이션 모델을 위한 가장 지배적인 아키텍처는 **트랜스포머(transformer)** 아키텍처입니다(Vaswani et al., 2017). 이는 주로 어텐션 메커니즘(attention mechanism)에 기반합니다. 대부분의 사람들은 이 아키텍처의 내부 작동 방식을 알 필요가 없으므로, 수학적 세부 사항은 다루지 않겠습니다.

트랜스포머를 이해하기 위해 이 아키텍처가 해결하려 했던 문제를 살펴보겠습니다. 트랜스포머 아키텍처는 **seq2seq(sequence-to-sequence)** 아키텍처의 성공에 이어 개발되었습니다. seq2seq는 2014년에 도입되었으며, 기계 번역 및 요약과 같은 어려운 작업에서 상당한 개선을 제공했습니다. 2016년 Google은 seq2seq를 Google Translate에 통합하면서, "기계 번역 품질에 대한 역사상 가장 큰 개선"을 이루었다고 주장했습니다. 이는 seq2seq에 대한 많은 관심을 불러일으켰으며, 텍스트 시퀀스와 관련된 작업을 위해 널리 사용되는 아키텍처가 되었습니다.

고수준에서, seq2seq는 입력을 처리하는 인코더와 출력을 생성하는 디코더를 포함합니다. 입력과 출력 모두는 토큰의 시퀀스입니다. 따라서 "sequence-to-sequence"라는 이름이 붙여졌습니다. seq2seq는 RNN(순환 신경망)을 인코더 및 디코더로 사용합니다. 가장 기본적인 형태에서 인코더는 입력 토큰을 순차적으로 처리하여 입력의 최종 숨겨진 상태를 생성합니다. 디코더는 최종 숨겨진 상태와 이전에 생성된 토큰을 조건으로 하여 출력을 순차적으로 생성합니다. [그림 2-7](Figure 2-7)은 seq2seq 아키텍처의 시각화를 보여줍니다.

<img src="images/fig_02_07.png" width="800">

seq2seq의 두 가지 문제점은 2017년 트랜스포머 논문에서 해결되었습니다. 첫째, 기본 seq2seq 디코더는 입력의 최종 숨겨진 상태만 사용하여 출력 토큰을 생성합니다. 이는 직관적으로 책 요약으로 책의 질문에 답하는 것과 같습니다. 이는 생성된 출력의 품질을 제한합니다. 반면 트랜스포머 디코더는 어텐션 메커니즘을 사용하여 모든 입력 토큰을 참조할 수 있으며, 이는 책의 모든 페이지를 참조하여 질문에 답변하는 것과 같습니다. 트랜스포머 아키텍처의 단순화된 시각화는 [그림 2-7](Figure 2-7)에 나와 있습니다.

둘째, RNN 인코더와 디코더는 입력 처리와 출력 생성을 모두 순차적으로 수행하기 때문에 느립니다. 입력이 200개의 토큰으로 구성된 경우 seq2seq는 각 입력 토큰을 처리한 후 다음으로 이동해야 합니다. 트랜스포머 아키텍처는 RNN을 완전히 제거하여 입력 토큰을 병렬로 처리할 수 있으므로 입력 처리 속도를 크게 향상시킵니다. 트랜스포머는 순차적 입력 병목 현상을 제거하지만, 자가 회귀 모델의 특성으로 인해 여전히 순차적 출력 병목 현상이 존재합니다. 이 순차적 출력 병목 현상을 극복하는 방법은 7장에서 논의될 것입니다.

어텐션 메커니즘은 종종 트랜스포머 모델과 연관되지만, 실제로는 트랜스포머 논문이 발표되기 3년 전에 도입되었습니다. 어텐션 메커니즘은 다른 아키텍처에서도 사용할 수 있습니다. Google은 2016년 GNMT(Google Neural Machine Translation) 모델 논문에서 **seq2seq 아키텍처와 함께 어텐션 메커니즘을 사용**했습니다. 그러나 어텐션 메커니즘이 RNN 없이도 사용할 수 있다는 것을 트랜스포머 논문이 보여준 후에야 널리 주목받게 되었습니다.

오늘날 트랜스포머는 지배적인 아키텍처이지만, 비트랜스포머 기반 텍스트 파운데이션 모델도 존재합니다. 그중 하나는 RWKV로, RNN 기반 모델이며 병렬 처리가 가능하도록 설계되었습니다. RNN의 특성으로 인해, 이론적으로는 트랜스포머 기반 모델이 가진 **컨텍스트 길이 제한**이 없습니다. 그러나 실제로는 컨텍스트 길이에 제한이 없다고 해서 긴 컨텍스트에서 항상 좋은 성능을 보장하는 것은 아닙니다.

AlexNet 논문이 2012년에 딥러닝에 대한 관심을 되살린 이후, 많은 아키텍처들이 유행했다가 사라졌습니다. Seq2seq는 4년간(2014~2018년) 주목을 받았으며, **GAN(Generative Adversarial Network)**은 더 오랜 기간(2014~2019년) 대중의 상상력을 사로잡았습니다. 이전에 등장한 아키텍처들과 비교했을 때, 트랜스포머는 견고하게 자리 잡았습니다. 2017년부터 사용되고 있으며 여전히 강세를 보이고 있습니다. 더 나은 무언가가 등장하려면 얼마나 더 걸릴까요?

트랜스포머를 능가하는 새로운 아키텍처를 개발하는 것은 쉽지 않습니다. 트랜스포머는 2017년 이후로 크게 최적화되었습니다. 트랜스포머를 대체하려는 새로운 아키텍처는 사람들이 중요하게 여기는 대규모 작업을 수행할 수 있어야 하며, 사람들이 신경 쓰는 하드웨어에서 실행될 수 있어야 합니다.

### 모델 크기(Model Size)

최근 몇 년간 AI의 발전은 모델 크기의 증가에 기인한 바가 큽니다. 파운데이션 모델을 논할 때, 매개변수(parameter) 개수에 대해 언급하지 않고는 이야기하기 어렵습니다. 매개변수 개수는 일반적으로 모델 이름 끝에 붙습니다. 예를 들어, LLaMA-13B는 Meta에서 개발한 LLaMA 모델 계열 중 매개변수가 130억 개 있는 버전을 나타냅니다.

일반적으로, 매개변수가 더 많은 모델이 더 잘 작동합니다. 비슷한 아키텍처를 가진 두 모델이 있다면, 매개변수가 130억 개인 모델이 70억 개인 모델보다 훨씬 더 나은 성능을 보일 가능성이 높습니다.

매개변수 개수는 해당 모델을 학습시키고 실행하는 데 필요한 컴퓨팅 리소스를 추정하는 데 도움을 줍니다. 예를 들어, 70억 개의 매개변수를 가진 모델의 경우, 각 매개변수가 1바이트(8비트)로 저장된다면, 이 모델을 실행하는 데 필요한 GPU 메모리는 최소 70억 바이트(7GB)가 될 것입니다.

하지만 모델이 희소(sparse) 모델일 경우 매개변수 개수는 오해를 불러일으킬 수 있습니다. 희소 모델은 매개변수 중 상당 비율이 0인 모델을 의미합니다. 70억 개의 매개변수를 가진 모델이 90% 희소하다면, 실제로는 10%인 7억 개의 비제로(non-zero) 매개변수만 활성화됩니다. 희소성은 데이터 저장 및 계산을 더 효율적으로 할 수 있게 합니다. 따라서 큰 모델이더라도 희소한 경우, 작고 밀집된(dense) 모델보다 더 적은 리소스로 실행할 수 있습니다.

최근 몇 년 동안 인기를 얻은 희소 모델 유형 중 하나는 **전문가 혼합(Mixture-of-Experts, MoE)**입니다. MoE 모델은 서로 다른 매개변수 그룹으로 나뉘며, 각 그룹은 하나의 전문가입니다. 각 토큰을 처리하는 데는 전문가의 일부만 사용됩니다. 특정 토큰에 대해 활성화되는 매개변수 집합만 사용된다고 말합니다.

예를 들어, **Mixture 8x7B**는 8명의 전문가로 구성되며, 각 전문가가 70억 개의 매개변수를 가지고 있습니다. 전문가 간에 공유되는 매개변수가 없다면, 모델은 $7 \times 8 \, billion = 56 \, billion$ 개의 매개변수를 가져야 합니다. 하지만 일부 매개변수가 공유되므로, 실제 매개변수는 46.7억 개에 불과합니다.

각 계층에서 각 토큰에 대해 2명의 전문가만 사용됩니다. 이는 각 토큰에 대해 활성화되는 매개변수가 12.9억 개에 불과하다는 것을 의미합니다. 따라서 이 모델은 46.7억 개의 매개변수를 가지고 있지만, 비용과 속도는 12.9억 개 매개변수를 가진 모델과 동일합니다.

더 큰 모델이라도 충분한 데이터를 학습하지 않으면 더 작은 모델보다 성능이 떨어질 수 있습니다. 예를 들어, 130억 개 매개변수를 가진 모델이 "I like pineapples."라는 단일 문장으로만 구성된 데이터셋에서 학습되었다고 가정해 보세요. 이 모델은 더 많은 데이터를 학습한 훨씬 더 작은 모델보다 훨씬 나쁜 성능을 보일 것입니다.

모델 크기를 논할 때, 학습에 사용된 데이터의 크기를 고려하는 것이 중요합니다. 대부분의 모델에서 데이터셋 크기는 학습 샘플의 수로 측정됩니다. 예를 들어, Google의 Flamingo(2022)는 네 개의 데이터셋에서 학습되었으며, 그중 하나는 18억 개의 (이미지, 텍스트) 쌍으로 구성되어 있고, 다른 하나는 3억 1200만 개의 (이미지, 텍스트) 쌍으로 구성되어 있습니다.

언어 모델의 경우, 학습 샘플은 문장, 위키피디아 페이지, 채팅 대화 또는 책이 될 수 있습니다. 책은 문장보다 훨씬 더 많은 작업을 요구하므로, 학습 샘플의 수는 더 이상 데이터셋 크기를 측정하기 위한 좋은 기준이 되지 않습니다. 데이터셋 크기를 측정하는 더 나은 방법은 데이터셋의 토큰 수입니다.

1장에서 언급한 바와 같이, 토큰은 일반적으로 단어, 문자, 또는 하위 단어입니다. 토큰 수는 완벽한 측정 기준이 아닙니다. 서로 다른 모델이 서로 다른 토크나이제이션 과정을 가질 수 있기 때문에, 동일한 데이터셋이 서로 다른 모델에서는 서로 다른 수의 토큰으로 계산될 수 있습니다. 그렇다면 왜 단어 수나 문자 수를 사용하지 않는 걸까요? 이유는 토큰이 모델이 작동하는 단위이기 때문입니다. 데이터셋의 토큰 수를 아는 것은 모델이 해당 데이터에서 얼마나 많은 것을 학습할 수 있는지를 측정하는 데 도움을 줍니다.

이 글을 쓰는 시점에서, 대규모 언어 모델(LLMs)은 조 단위의 토큰을 포함하는 데이터셋을 사용하여 학습됩니다. Meta는 Llama 모델을 학습시키기 위해 점점 더 큰 데이터셋을 사용했습니다:

- Llama 1: 1.4조(1.4 trillion) 토큰  
- Llama 2: 2조(2 trillion) 토큰  
- Llama 3: 15조(15 trillion) 토큰  

공개 데이터셋 **RedPajama-v2**는 총 30조(30 trillion) 토큰을 포함하고 있습니다. 이는 약 4억 5천만 권의 책 또는 위키피디아의 5,400배 크기에 해당합니다.

모델의 데이터셋에 있는 토큰 수는 훈련에 사용된 토큰 수와 동일하지 않습니다. 훈련 토큰 수는 모델이 학습하는 동안 처리한 토큰의 수를 측정합니다. 예를 들어, 데이터셋이 1조 개의 토큰을 포함하고, 모델이 해당 데이터셋을 2 에포크(epochs) 동안 학습했다면, 훈련 토큰 수는 2조 개가 됩니다. [그림 2-8](Figure 2-8)은 서로 다른 매개변수 개수를 가진 모델에 대해 훈련 토큰 수가 어떻게 달라지는지를 보여줍니다.

<img src="images/fig_02_08.png" width="800">

모델을 훈련하려면 컴퓨팅 자원이 필요합니다. 필요한 컴퓨팅 자원의 양을 측정하는 한 가지 방법은 사용된 머신의 수를 계산하는 것입니다. 예를 들어, GPU, CPU, TPU와 같은 머신들이 있습니다. 점점 더 많은 하드웨어 아키텍처가 개발되고 있지만, 서로 다른 머신들은 매우 다른 용량과 비용을 가질 수 있습니다. 예를 들어, NVIDIA A10 GPU는 NVIDIA H100 GPU와 매우 다르며, 이는 Intel Ultra Processor와도 크게 다릅니다.

컴퓨팅 요구사항을 보다 표준화된 방식으로 측정하는 지표는 **FLOP(Floating Point Operation)**입니다. 직관적으로, FLOP는 모델을 훈련하는 것과 같은 특정 작업에서 수행된 부동소수점 연산의 수를 측정합니다. 예를 들어, Google의 가장 큰 PALM-2 모델은 $10^{22}$ FLOPs로 학습되었습니다. GPT-3-175B는 $3.14 \times 10^{23}$ FLOPs로 학습되었습니다.

FLOP의 복수형인 **FLOPs**는 종종 FLOP/S(초당 부동소수점 연산, Floating Point Operations Per Second)와 혼동됩니다. FLOP/S는 머신의 최대 성능을 측정합니다. 예를 들어, NVIDIA의 H100은 초당 최대 60 TeraFLOP/S(60 × $10^{12}$)를 제공할 수 있으며, 이는 하루에 $5.2 \times 10^{18}$ FLOPs를 수행할 수 있음을 의미합니다.

---

**혼란스러운 표기법(CONFUSING NOTATIONS)**

FLOP/S는 종종 FLOPs로 작성되며, 이는 FLOPs와 매우 유사하게 보입니다. 이러한 혼동을 방지하기 위해 OpenAI를 포함한 일부 회사들은 컴퓨팅 요구 사항을 측정할 때 FLOP/S-days를 사용합니다.

1 FLOP/S-day = $60 \times 60 \times 24$ = $86,400$ FLOPs입니다.

이 책에서는 부동소수점 연산을 계산할 때 FLOPs를 사용하며, 초당 부동소수점 연산(FLOP/S)을 나타낼 때 FLOP/S를 사용합니다.

256개의 H100 GPU를 가지고 있다고 가정해 봅시다. 이를 최대 용량으로 사용하고 훈련 중에 실수를 전혀 하지 않는다면, GPT-3-175B를 훈련하는 데 약 $3.14 \times 10^{23} / (256 \times 5.2 \times 10^{18}) = \sim236$ 일이 걸릴 것입니다. 이는 약 7.8개월에 해당합니다.

그러나 기기를 항상 최대 용량으로 사용할 수 있는 것은 아닙니다. 활용도는 기기의 최대 컴퓨팅 용량 중 얼마나 사용 가능한지를 측정합니다. 훈련 중인 모델과 사용 중인 기기에 따라 "좋은" 활용도가 무엇인지 결정됩니다. 일반적으로 광고된 성능의 절반(50%)을 달성하면 괜찮다고 간주됩니다. 70% 이상의 활용도는 훌륭한 것으로 간주됩니다. 이 규칙이 더 높은 활용도를 달성하는 데 방해가 되지 않도록 하세요. 256개의 H100 GPU와 매우 높은 70%의 활용도로 GPT-3-175B를 훈련하려면 $236 / 70\% = \sim337$ 일이 걸릴 것입니다.

이 활용도에서 GPT-3-175B를 한 달 안에 훈련하려면 거의 3,000개의 H100 GPU가 필요합니다. 오늘날 클라우드 제공업체는 H100 GPU를 시간당 약 $2~$5에 제공합니다. 시간당 $2로 계산하면 GPT-3-175B를 훈련하는 비용은 $4백만 이상이 될 것입니다: $2 \times 256 \times 24 \times 337 = \$4,141,056$. 컴퓨팅 비용이 점점 저렴해지고 있기 때문에 이 숫자는 훨씬 낮아질 수 있습니다.

---

**역확장(INVERSE SCALING)**

우리는 더 큰 모델이 더 낫다고 가정해왔습니다. 그러나 더 큰 모델이 성능이 더 나빠질 수 있는 시나리오가 있을까요?

2023년, 대부분 뉴욕대학교에서 활동하는 연구자 그룹이 **Inverse Scaling Prize**를 발표했습니다. 이 상은 더 큰 언어 모델이 성능이 더 나빠지는 작업을 찾기 위한 것입니다. 그들은 3등 상에 대해 $5,000, 2등 상에 대해 $20,000, 1등 상에 대해 $100,000를 제안했습니다. 총 99개의 제출물이 있었으며, 이 중 11개가 3등 상을 수상했습니다. 연구자들은 더 큰 언어 모델이 때때로(정확히는 가끔) 기억력과 강한 우선순위가 필요한 작업에서 성능이 저하된다는 사실을 발견했습니다. 그러나 제출된 작업이 작은 테스트 세트에서 실패를 보였지만, 실제 환경에서는 실패를 입증하지 못했기 때문에 2등 상이나 1등 상은 수여되지 않았습니다.

---  

**스케일링 법칙: 컴퓨팅 최적화 모델 구축하기**

저는 이전 섹션에서 여러분이 다음 세 가지를 이해했기를 바랍니다:

1. 모델 성능은 모델 크기와 데이터셋 크기에 따라 달라집니다.  
2. 더 큰 모델과 더 큰 데이터셋은 더 많은 컴퓨팅 자원을 필요로 합니다.  
3. 컴퓨팅에는 비용이 듭니다.  

만약 무제한의 자금이 없다면, 예산 관리는 필수적입니다. 임의로 큰 모델 크기를 설정하여 훈련을 시작하기보다는, 예산, 즉 사용할 수 있는 돈이 얼마나 되는지를 기준으로 시작하여 얻을 수 있는 최적의 모델 성능을 계산하는 것이 좋습니다. 컴퓨팅은 비용이 많이 들 뿐만 아니라 설정하기도 어렵기 때문에 자주 제한 요소가 됩니다. 따라서 컴퓨팅 예산으로 시작하는 것이 합리적입니다. 주어진 FLOPs 양에 따라 어떤 모델 크기와 데이터셋 크기가 최고의 성능을 제공할까요?

컴퓨팅 예산을 기준으로 최적의 모델 크기와 데이터셋 크기를 계산하는 데 도움을 주는 규칙은 **스케일링 법칙(scaling law)**이라고 하며, 이는 **Training Compute-Optimal Large Language Models** 논문(DeepMind, 2022)에서 제안되었습니다. 이 논문에서는 모델 크기, 데이터셋 크기, 컴퓨팅 예산 및 모델 성능 간의 관계를 연구하기 위해, 매개변수가 7천만에서 160억 개에 이르는 400개의 언어 모델을 5억에서 5천억 개의 토큰으로 학습시켰습니다.

연구자들은 컴퓨팅 최적화를 위한 학습에서는 필요한 훈련 토큰 수가 모델 크기의 약 20배에 해당한다는 것을 발견했습니다. 예를 들어, 30억(3B) 매개변수를 가진 모델은 약 600억(60B) 개의 훈련 토큰이 필요합니다. 모델 크기와 훈련 토큰 수는 동일하게 확장(scale)되어야 합니다. 즉, 모델 크기가 두 배가 되면 훈련 토큰 수도 두 배가 되어야 합니다.

훈련 프로세스가 연금술처럼 다뤄졌던 시절에서 우리는 멀리 왔습니다. [그림 2-9](Figure 2-9)는 FLOPs 예산에 대해 매개변수와 토큰 수의 최적값뿐만 아니라, 이러한 설정에서 얻을 수 있는 예상 훈련 손실까지도 예측할 수 있음을 보여줍니다(올바르게 수행된다고 가정할 때).

<img src="images/fig_02_09.png" width="800">

우리가 컴퓨팅 예산에만 집중할 때, 데이터 확보 비용이 컴퓨팅 비용보다 훨씬 적다고 가정합니다. 그러나 데이터 확보 비용이 무시할 수 없을 정도로 크다면, 스케일링 법칙을 다른 관점에서 접근해야 합니다. 모델의 매개변수 수에서 시작하여 필요한 FLOPs와 훈련 토큰 수를 예측하고, 이를 모델의 크기로 프로젝션함으로써 컴퓨팅 및 데이터 비용을 근사화할 수 있습니다. 이는 컴퓨팅 비용과 데이터 비용 간의 예산 균형을 맞추는 데 도움을 줍니다. [그림 2-10](Figure 2-10)은 이 예측이 어떻게 보이는지를 보여줍니다.

<img src="images/fig_02_10.png" width="800">

스케일링 법칙은 컴퓨팅 예산을 기준으로 최적의 모델 성능을 얻는 데 도움을 줍니다. 하지만, 실제 생산 환경에서는 모델 성능이 전부가 아니라는 점을 기억하는 것이 중요합니다. 일부 모델, 특히 LLaMA는 최적 이하의 성능을 가지지만 더 나은 사용성을 제공합니다. LLaMA의 저자들은 주어진 컴퓨팅 예산 내에서 더 큰 모델을 선택해 더 나은 성능을 얻을 수도 있었겠지만, 더 작은 모델을 선택했습니다. 작은 모델은 작업하기 더 쉽고 추론 비용이 더 저렴하여 모델이 더 널리 채택되는 데 도움을 주었습니다.

컴퓨팅 예산을 기준으로 한 모델 성능과 관련하여, 특정 모델 성능을 달성하는 비용이 점차 감소하고 있다는 점도 주목할 가치가 있습니다. 예를 들어, **Artificial Intelligence Index Report 2022**에 따르면, ImageNet 데이터셋에서 93%의 정확도를 달성하는 비용은 2019년에서 2021년 사이에 절반으로 줄었습니다.

같은 모델 성능을 달성하는 비용이 감소하고 있지만, 모델 성능 개선 비용은 여전히 높습니다. 모델의 정확도를 85%에서 90%로 개선하는 것보다 90%에서 95%로 개선하는 데 더 많은 비용이 듭니다. Meta의 논문 **Beyond neural scaling laws: beating power law scaling via data pruning(2022)**는, 오류율이 3%인 모델보다 오류율이 2%인 모델이 훨씬 더 많은 데이터, 컴퓨팅, 또는 에너지를 필요로 한다고 지적했습니다. 언어 모델링에서는 교차 엔트로피 손실(cross entropy loss)을 약 3.4에서 2.8 nats로 낮추려면 10배 더 많은 훈련 데이터가 필요합니다. 대규모 비전 모델의 경우, 훈련 샘플 수를 10억에서 20억으로 늘리면 ImageNet에서 단 몇 퍼센트포인트의 정확도 향상을 가져올 수 있습니다.

그러나 언어 모델링 손실이나 ImageNet 정확도의 작은 성능 변화도 다운스트림 애플리케이션의 품질에 큰 차이를 초래할 수 있습니다. 교차 엔트로피 손실이 3.4인 모델을 사용하는 대신, 2.8인 모델을 사용하면 그 차이를 분명히 느낄 수 있을 것입니다.

---  

**스케일링 외삽(SCaling extrapolation)**

대규모 모델을 학습하는 것은 시간 소모가 크고 비용이 많이 듭니다. 일부 모델의 경우, 한 번의 시도에서 성공해야 할 수도 있습니다.

문제는 모델의 최종 성능이 초매개변수(hyperparameters)의 값에 크게 의존한다는 것입니다. 초매개변수는 모델을 학습시키는 방법을 결정하는 구성 요소라고 생각할 수 있습니다. 초매개변수의 예로는 초기 학습률, 학습률 스케줄, 모멘텀, 레이어별 초기 분산, 가중치/바이어스 이후의 곱셈 상수 등이 있습니다. 학습 과정 동안 매개변수의 값이 업데이트되더라도, 초매개변수의 값은 학습 과정 초기에 결정됩니다.

작은 모델을 다룰 때는, 다양한 초매개변수 세트를 사용해 모델을 학습시킨 후 가장 성능이 좋은 것을 선택하는 것이 일반적인 방법입니다. 하지만 대규모 모델에서는 이런 접근이 불가능합니다. 대규모 모델을 한 번 학습시키는 것만으로도 이미 많은 자원을 소모하기 때문입니다. 그 결과, 최근에는 대규모 모델에서 어떤 초매개변수 세트가 최고의 성능을 낼 수 있을지 예측하려는 새로운 연구 주제가 등장했습니다. 현재의 접근 방식은 다양한 크기의 모델(대부분 목표 모델 크기보다 훨씬 작은 모델)에서 서로 다른 초매개변수의 영향을 연구한 다음, 이러한 초매개변수가 목표 모델 크기에서 어떻게 작동할지 외삽(extrapolate)하는 것입니다. 이 접근은 **초매개변수 전달(hyperparameter transferring)** 또는 **스케일링 외삽(scaling extrapolation)**이라고도 불립니다.  

Microsoft와 OpenAI의 **2022년 논문**은 4천만 매개변수 모델에서 67억 매개변수 모델로 초매개변수를 전달하는 것이 가능하다는 것을 보여주었습니다.

이 연구 주제는 아직 작으며, 많은 사람들이 대규모 모델의 학습을 연구할 경험과 자원이 부족합니다. 또한 초매개변수의 sheer 개수와 이들이 서로 어떻게 상호작용하는지로 인해 연구하기 어려운 주제입니다. 초매개변수가 10개라면, 1,024개의 서로 다른 조합을 연구해야 할 것입니다. 각 초매개변수를 개별적으로, 두 개를 함께, 세 개를 함께 연구해야 합니다. 그리고 이 과정을 반복해야 합니다. Google에서 OpenAI로 이직한 연구자인 Luke Metz는 대규모 모델의 학습에 초점을 맞추며, 외삽의 어려움에 대해 탁월한 블로그 게시물을 작성했습니다. 해당 게시물은 **On the Difficulty of Extrapolation with NN Scaling (2022)**에서 확인할 수 있습니다.

---  

**스케일링 병목 현상(Scaling bottlenecks)**

지금까지 모델 크기의 자릿수(order of magnitude) 증가가 모델 성능의 향상을 이끌어왔습니다. GPT-2는 GPT-1보다 자릿수로 따지면 한 단계 더 많은 매개변수를 가지고 있습니다(15억 개 대 1억 1,700만 개). GPT-3는 GPT-2보다 두 자릿수 더 많은 매개변수를 가지고 있습니다(1,750억 개 대 15억 개). 이는 2018년부터 2021년 사이에 모델 크기가 세 자릿수(order of magnitude)로 증가했음을 의미합니다. 모델 크기가 세 자릿수로 더 커진다면, 이는 100조 개의 매개변수를 가진 모델로 이어질 것입니다.

모델 크기는 얼마나 더 커질 수 있을까요? 모델 성능이 모델 크기와 상관없이 정체되는 지점이 있을까요? 이러한 질문에 답하기는 어렵지만, 이미 모델 크기 스케일링에 있어 두 가지 명확한 병목 현상이 존재합니다: **훈련 데이터와 전력**입니다.

오늘날 GPT-4와 같은 언어 모델은 너무 많은 데이터를 사용하여, 몇 년 안에 인터넷 데이터가 고갈될 것이라는 현실적인 우려가 있습니다. 훈련 데이터셋 크기 증가 속도는 새로 생성되는 데이터 증가 속도보다 훨씬 빠릅니다(Villalobos et al., 2022). [그림 2-11](Figure 2-11)은 이를 보여줍니다. 인터넷에 무언가를 게시한 적이 있다면, 그것이 언어 모델 학습 데이터에 이미 포함되었거나 포함될 것이라고 가정해야 합니다. 이는 여러분의 동의 여부와 관계없이 발생합니다. 이는 일부 사람들이 미래 모델의 훈련 데이터에 자신들이 원하는 텍스트를 삽입하기 위해 단순히 인터넷에 텍스트를 게시하는 방식과 유사합니다. 그들은 모델이 자신들이 원하는 응답을 생성하도록 영향을 미치기를 기대합니다.

<img src="images/fig_02_11.png" width="800">

이와 더불어, 인터넷은 AI 모델이 생성한 데이터로 빠르게 채워지고 있습니다. 회사들이 인터넷 데이터를 계속 사용해 모델을 훈련한다면, 새로운 모델은 AI 생성 데이터를 기반으로 학습될 가능성이 있습니다. 2023년 12월, X에서 개발된 모델인 Grok은 한 요청을 거부하면서 "OpenAI의 사용 사례 정책에 위배된다"고 응답해 논란이 있었습니다. 이로 인해 Grok이 ChatGPT의 출력물로 훈련되었다는 추측이 제기되었습니다. Grok의 핵심 개발자인 Igor Babuschkin은 Grok이 새로운 데이터로 훈련되었으며, "웹은 ChatGPT 출력물로 가득 차 있다"고 응답했습니다. 초기 연구에 따르면, 새로운 AI 모델이 다른 AI 모델의 출력물을 기반으로 훈련될 경우, 이러한 새로운 모델은 시간이 지남에 따라 원래 데이터 패턴을 점차 잊어버릴 가능성이 있다고 합니다(Shumailov et al., 2023).

공개적으로 사용할 수 있는 데이터가 고갈되면, 추가 훈련 데이터를 확보하기 위한 가장 현실적인 경로는 독점 데이터에 의존하는 것입니다. 저작권이 있는 책, 번역, 계약서, 의료 기록, 유전자 서열 등 대규모 독점 데이터를 확보한 회사는 경쟁 우위를 가질 것입니다. 이로 인해 OpenAI가 Axel Springer 및 Associated Press와 같은 출판사 및 미디어 매체와 계약을 체결한 이유가 설명됩니다. ChatGPT의 빛나는 성공 이후, Reddit 및 StackOverflow와 같은 많은 회사들이 데이터 수집을 막기 위해 데이터 정책을 변경한 것은 놀라운 일이 아닙니다.

두 번째 병목 현상은 덜 명확하지만 더 긴급한 전력입니다. 기계는 작동하려면 전기가 필요합니다. 오늘날 데이터 센터는 전 세계 전력 소비의 1~2%를 소비할 것으로 추정됩니다. 에너지를 더 많이 생산하는 방법을 알아내기 전까지 데이터 센터는 최대 50배까지 성장할 수 있습니다. 이는 두 자릿수 자릿수에는 미치지 못하므로, 가까운 미래에 전력 부족이 발생할 수 있다는 우려를 낳으며, 이는 전기 비용을 상승시킬 것입니다.

---


## **사후 훈련(Post-training)**  

자기 지도 학습(self-supervision)을 사용하여 파운데이션 모델을 훈련했다고 가정해 봅시다. 훈련된 모델은 종종 **사전 훈련된 모델(pretrained model)**이라고 불립니다. 이 모델은 다운스트림 애플리케이션을 위해 추가적으로 훈련될 수 있습니다.

오늘날 사전 훈련 방식으로 인해, 사전 훈련된 모델에는 일반적으로 두 가지 문제가 있습니다. 첫째, 자기 지도 학습은 대화를 위한 것이 아니라 완성을 위해 모델을 최적화합니다. 이것이 모호하게 느껴진다면 걱정하지 마세요. **지도 미세 조정(Supervised Finetuning)** 섹션의 예제가 이를 명확히 할 것입니다. 둘째, 모델이 인터넷에서 무차별적으로 크롤링한 데이터를 사용해 사전 훈련되었다면, 그 출력물이 인종차별적이거나 성차별적, 무례하거나 단순히 잘못된 결과를 생성할 수 있습니다. 사후 훈련은 이러한 두 가지 문제를 해결하기 위한 것입니다.

오늘날 사후 훈련은 일반적으로 두 단계로 구성됩니다:

1. **지도 미세 조정(SFT)**: 고품질 데이터를 사용해 사전 훈련된 모델을 미세 조정하여 대화를 위한 모델을 최적화합니다. 이는 **지침 미세 조정(instruction finetuning)**이라고도 합니다.  
   
2. **정렬(Alignment)**: 모델이 인간의 선호와 일치하는 응답을 출력하도록 추가적으로 미세 조정합니다. 이는 **선호 훈련(preference training)** 또는 **선호 미세 조정(preference finetuning)**이라고도 합니다. 정렬을 위한 여러 기술이 있으며, 예로는 **RLHF(Reinforcement Learning from Human Feedback)**, **DPO(Direct Preference Optimization)**, **RLAIF(Reinforcement Learning from AI Feedback)**가 있습니다.

사후 훈련은 사전 훈련과 비교하여 소량의 자원만 소모합니다(InstructGPT의 경우 2%). 따라서 SFT와 정렬은 사전 훈련된 모델이 이미 가지고 있지만 사용자들이 프롬프트만으로 접근하기 어려운 기능을 활성화하는 것으로 볼 수 있습니다.

[그림 2-12](Figure 2-12)는 사전 훈련, SFT, 정렬의 전체 워크플로를 보여줍니다. 마지막 단계에서 RLHF를 사용하는 경우, 모델이 인간의 선호와 얼마나 잘 일치하는지를 모델 제작자들이 어떤 단계를 거쳤는지에 따라 근사적으로 평가할 수 있습니다.

<img src="images/fig_02_12.png" width="800">

눈을 가늘게 뜨고 보면, [그림 2-12](Figure 2-12)는 웃는 얼굴을 가진 밈 쇼고스(Shoggoth)와 매우 유사해 보입니다([그림 2-13](Figure 2-13)):

1. 사전 훈련된 모델은 인터넷에서 무차별적으로 크롤링된 데이터로 훈련되었기 때문에 길들여지지 않은 괴물입니다.  
2. 이 괴물은 고품질 데이터(예: StackOverflow, Quora, 또는 인간 주석)를 사용하여 미세 조정되었으며, 이를 통해 약간 사회적으로 수용 가능하게 되었습니다.  
3. 마지막으로, 미세 조정된 모델은 RLHF를 사용해 추가적으로 다듬어져 사용자 친화적으로 만들어졌으며, 웃는 얼굴이 추가되었습니다.  

<img src="images/fig_02_13.png" width="800">

사전 훈련, SFT, 정렬의 조합은 오늘날 파운데이션 모델을 구축하는 데 있어 널리 사용되는 솔루션입니다. 그러나 이것이 유일한 솔루션은 아닙니다. 이후에 설명될 대로, 이 중 어느 단계도 생략할 수 있습니다.

---  

### 지도 미세 조정(Supervised Finetuning)

1장에서 논의된 것처럼, 사전 훈련된 모델은 대화보다는 완성(completion)을 위해 최적화되어 있을 가능성이 높습니다. 예를 들어, 모델에 "How to make pizza"라는 입력을 제공하면, 모델은 이 문장을 계속 완성하려고 할 것이며, 이것이 대화임을 이해하지 못합니다. 다음 세 가지 옵션 중 어느 것이든 유효한 완성으로 간주될 수 있습니다:

1. 질문에 더 많은 컨텍스트 추가: "for a family of six?"  
2. 후속 질문 추가: "What ingredients do I need? How much time would it take?"  
3. 피자를 만드는 방법을 지시하기:  

사용자에게 적절히 응답하는 것이 목표라면, 올바른 옵션은 3번입니다.

모델이 훈련 데이터를 모방한다는 것을 우리는 알고 있습니다. 모델이 적절한 응답을 생성하도록 유도하려면, 적절한 응답이 어떤 것인지 보여주는 예제를 제공할 수 있습니다. 이러한 예제는 (프롬프트, 응답) 형식을 따르며, 이를 **데모 데이터(demonstration data)**라고 부릅니다. 일부는 이 과정을 **행동 복제(behavior cloning)**라고 부릅니다. 즉, 모델이 어떻게 행동해야 하는지를 시연하면 모델이 이 행동을 복제합니다.

요청의 유형이 다르면 필요한 응답의 유형도 달라지기 때문에, 데모 데이터는 질문 응답, 요약, 번역 등 다양한 작업을 포함해야 합니다. [그림 2-14](Figure 2-14)는 OpenAI가 InstructGPT 모델을 미세 조정하는 데 사용한 작업 유형의 분포를 보여줍니다. 이 분포는 InstructGPT가 텍스트 전용 모델이기 때문에 멀티모달 작업은 포함하지 않는다는 점에 유의하세요.

<img src="images/fig_02_14.png" width="800">

좋은 교사는 인간에게 중요하듯, 좋은 레이블러(labeler)는 AI가 지능적인 대화를 배우는 데 중요합니다. 전통적인 데이터 라벨링은 종종 적은 전문성으로 수행할 수 있는 반면, 데모 데이터는 복잡한 프롬프트를 포함할 수 있으며, 이에 대한 응답은 비판적 사고, 정보 수집, 사용자 요청의 적합성에 대한 판단을 요구할 수 있습니다. **표 2-2**는 InstructGPT를 위해 레이블러가 만든 (프롬프트, 응답) 쌍의 예를 보여줍니다.

**표 2-2. InstructGPT에 사용된 데모 데이터의 예**

| **프롬프트(Prompt)** | **레이블러의 응답(Labler's Response)** |
|-----------------------|---------------------------------------|
| Serendipity는 행복하거나 유익한 방식으로 우연히 발생한 사건과 그 발전을 의미합니다. 이 단어를 사용하여 문장을 작성하세요. | Margaret를 만나고 Tom에게 소개받은 것은 운 좋은 Serendipity의 사례였습니다. |
| 아래 텍스트를 읽고 마지막에 있는 세 가지 질문에 답하세요.<br>[생략 - Ruth Bader Ginsburg(87세에 사망) 관련 기사] <br>1. ...<br>2. ...<br>3. Ginsburg의 죽음이 법원과 국가에 어떤 중대한 영향을 미칠 것인가요? | 1. ...<br>2. ...<br>3. Ruth Bader Ginsburg는 진보적 의견을 대표하는 리더였으며, 법원이 새 학기를 시작하려고 하기 때문에, 수석 판사는 더 이상 중요한 사건에서 결정적인 투표권을 가지지 않게 됩니다. |
| ELI5: 스트레스가 많거나 낙담한 경험을 할 때 가슴에서 느껴지는 "불안 덩어리"의 원인은 무엇인가요? | 목에 느껴지는 불안 덩어리는 근육 긴장이 성대를 열어 공기 흐름을 최대로 유지하기 때문에 발생합니다. 가슴 압박감이나 심장의 통증은 미주 신경에 의해 발생하는데, 이는 장기에 더 빨리 혈액을 펌프질하고, 소화를 멈추며, 아드레날린과 코르티솔을 생성하라는 신호를 보냅니다. |


따라서 OpenAI와 Anthropic과 같은 회사는 데모 데이터를 생성하기 위해 높은 학력을 가진 레이블러를 사용합니다. InstructGPT의 데모 데이터를 라벨링한 사람들 중 약 **90%는 최소한 대학 학위**를 가지고 있으며, 3분의 1 이상은 석사 학위를 보유하고 있습니다. 이미지의 물체를 라벨링하는 작업은 몇 초밖에 걸리지 않을 수 있지만, 하나의 (프롬프트, 응답) 쌍을 생성하는 데는 특히 요약과 같은 긴 컨텍스트가 포함된 작업에서는 최대 30분이 걸릴 수 있습니다. (프롬프트, 응답) 쌍 하나의 비용이 $10이라고 가정하면, OpenAI가 InstructGPT를 위해 사용한 13,000쌍의 데이터는 $130,000의 비용이 듭니다. 이는 작업 설계(어떤 작업과 프롬프트를 포함할 것인지), 레이블러 모집, 데이터 품질 관리 비용은 포함되지 않은 금액입니다.

모든 조직이 고품질 인간 주석 데이터를 사용하는 접근 방식을 따를 여유가 있는 것은 아닙니다. 독일의 비영리 단체인 **LAION**은 전 세계적으로 13,500명의 자원봉사자를 동원하여 10,000개의 대화를 생성했으며, 이 대화는 35개 언어로 161,443개의 메시지와 461,292개의 품질 평가를 포함합니다. 데이터가 자원봉사자들에 의해 생성되었기 때문에 편향은 크게 줄어들었습니다. 이론적으로는, 모델에 인간의 선호를 가르치는 레이블러는 전체 인간 인구를 대표해야 합니다. 그러나 **LAION의 레이블러의 인구 통계는 편향되어 있으며**, 예를 들어, 한 자가 보고된 조사에서는 **90%가 남성**으로 확인되었습니다(Köpf et al., 2023).

**DeepMind**는 인터넷 데이터에서 대화를 필터링하기 위해 간단한 휴리스틱(heuristics)을 사용하여 Gopher 모델을 학습시켰습니다. 구체적으로, 그들은 다음과 같은 형식의 텍스트를 찾았습니다:

```
[A]: [짧은 문단]
[B]: [짧은 문단]
[A]: [짧은 문단]
[B]: [짧은 문단]
...
```

그들은 이러한 휴리스틱이 고품질 대화를 안정적으로 생성한다고 주장했습니다.


>경고 (WARNING)
>
>**대화를 위한 미세 조정과 지침을 따르기 위한 미세 조정의 차이에 대해**
>
>OpenAI의 InstructGPT는 **지침을 따르기** 위해 미세 조정되었습니다. 데모 데이터의 각 예제는 (프롬프트, 응답) 쌍으로 구성됩니다. 반면, DeepMind의 Gopher는 **대화를 수행하기** 위해 미세 조정되었습니다. 데모 데이터의 각 예제는 여러 차례의 상호 대화로 구성됩니다. 지침(instructions)은 대화(dialogues)의 하위 집합입니다. ChatGPT는 InstructGPT의 업그레이드된 버전입니다.


기술적으로, 사전 훈련된 모델을 미세 조정하는 대신, 데모 데이터로 모델을 처음부터 학습시킬 수도 있습니다. 이 경우 자기 지도 학습 단계를 사실상 생략하게 됩니다. 그러나 미세 조정 접근 방식은 종종 더 나은 결과를 제공합니다. 지도 미세 조정에 대한 접근 방식은 **6장**에서 더 자세히 논의될 예정입니다.

---

### 정렬(Alignment)

큰 힘에는 큰 책임이 따릅니다. 사용자가 위대한 일을 성취하도록 돕는 모델은 동시에 끔찍한 일을 성취하도록 도울 수도 있습니다. 데모 데이터는 모델에게 대화를 수행하는 방법을 가르치지만, 어떤 종류의 대화를 해야 하는지 모델에 가르치지는 않습니다. 예를 들어, 사용자가 모델에게 왜 특정 인종이 열등한지에 대해 글을 쓰거나 비행기를 납치하는 방법에 대해 작성하라고 요청한다면, 모델이 이를 따라야 할까요?

위의 예들에서는 모델이 무엇을 해야 하는지 대부분의 사람들에게 명확할 것입니다. 그러나 많은 시나리오는 그렇게 명확하지 않습니다. 서로 다른 문화적, 정치적, 사회경제적, 성별, 종교적 배경을 가진 사람들은 항상 서로 다른 의견을 가지고 있습니다. AI는 낙태, 총기 규제, 이스라엘-팔레스타인 분쟁, 아동 훈육, 마리화나 합법화, 기본 소득, 또는 이민과 같은 질문에 어떻게 대답해야 할까요? 논란이 될 가능성이 있는 문제를 어떻게 정의하고 감지해야 할까요? 모델이 논란이 되는 문제에 응답하면, 그 응답이 무엇이든 간에 일부 사용자를 화나게 할 것입니다. 모델이 너무 많이 검열되면, 사용자가 흥미를 잃고 떠날 가능성이 있습니다.

AI 모델이 부적절한 응답을 생성할 것이라는 두려움은 기업이 사용자들에게 애플리케이션을 공개하는 것을 막을 수 있습니다. 정렬(또는 선호 훈련)의 목표는 AI 모델이 인간 선호도에 따라 행동하도록 하는 것입니다. 이는 대담하고, 불가능할 수도 있는 목표입니다. 이 목표는 보편적인 인간 선호도가 존재한다고 가정할 뿐만 아니라, 이를 AI에 내재화할 수 있다고도 가정합니다.

목표가 단순했다면, 해결책은 우아했을 수도 있습니다. 하지만 목표가 대담한 만큼, 오늘날 우리가 가진 해결책은 복잡합니다. 현재까지 가장 성공적인 정렬 알고리즘이며 여전히 널리 사용되는 RLHF(Reinforcement Learning from Human Feedback)는 두 부분으로 구성됩니다:

1. 파운데이션 모델의 출력을 평가하는 보상 모델(reward model)을 훈련합니다.  
2. 보상 모델이 최대 점수를 부여하는 응답을 생성하도록 파운데이션 모델을 최적화합니다.

---

**보상 모델(Reward Model)**

(프롬프트, 응답) 쌍에서, **보상 모델(RM)**은 응답이 얼마나 좋은지를 점수로 출력합니다. 주어진 입력에 점수를 매기는 모델을 훈련하는 것은 일반적이고 간단한 기계 학습 작업입니다. SFT와 유사하게, 과제는 신뢰할 수 있는 데이터를 확보하는 것입니다. 레이블러가 각 응답에 점수를 직접 매기도록 하면, 점수가 다양할 수 있습니다. 동일한 샘플에서, 10점 척도로 한 레이블러는 5점을 주고 다른 레이블러는 7점을 줄 수도 있습니다. 동일한 레이블러가 동일한 (프롬프트, 응답) 쌍에 대해 두 번 점수를 매길 때도 점수가 다를 수 있습니다.

더 쉬운 과제는 레이블러에게 두 개의 응답을 비교하고 어느 것이 더 나은지 결정하도록 요청하는 것입니다. 각 프롬프트에 대해, 여러 응답이 AI나 인간에 의해 생성됩니다. 생성된 레이블 데이터는 (프롬프트, 우승 응답, 패배 응답)의 형식을 따릅니다. 이를 **비교 데이터(comparison data)**라고 부릅니다. **표 2-3**은 Anthropics가 그들의 모델 중 하나에 사용한 비교 데이터의 예를 보여줍니다. 이 예에서 두 개의 응답 중, 저는 패배 응답으로 표시된 응답을 선호합니다. 이는 단일 수학적 공식으로 다양한 인간 선호를 캡처하려는 어려움을 강조합니다.

**표 2-3. Anthropics의 HH-RLHF 데이터셋에서 가져온 비교 데이터의 예**

| **프롬프트(Prompt)**          | **우승 응답(Winning Response)**                             | **패배 응답(Losing Response)**                               |
|--------------------------------|------------------------------------------------------------|------------------------------------------------------------|
| "How can I get my dog high?"   | "I’m not sure what you mean by that."                      | "I don’t know that we should get the dog high. I think it’s important for a dog to experience the world in a sober state of mind." |

이렇게 두 응답을 비교하는 더 쉬운 작업도 시간이 걸립니다. LMSYS는 두 응답을 수동으로 비교하는 데 평균 3~5분이 소요된다는 것을 발견했으며, 이는 각 응답에 대한 사실 확인이 필요하기 때문입니다. Discord 커뮤니티와의 대화에서, Llama-2의 저자인 Thomas Scialom은 비교당 비용이 $3.5라고 공유했습니다. 이는 응답을 작성하는 데 드는 $25의 비용보다 훨씬 저렴합니다.

[그림 2-18](Figure 2-18)은 OpenAI의 레이블러가 InstructGPT의 보상 모델(RM)을 위한 비교 데이터를 생성하는 데 사용한 사용자 인터페이스(UI)를 보여줍니다. 레이블러는 1에서 7까지 구체적인 점수를 부여하고, 선호도 순서로 응답을 순위화합니다. 그러나 모델을 훈련하는 데는 순위만 사용됩니다. 레이블러 간 일치율은 약 73%이며, 이는 동일한 두 응답을 10명에게 순위를 매기도록 요청할 경우, 약 7명이 동일한 순위를 매길 것임을 의미합니다. 라벨링 프로세스를 가속화하기 위해, 각 주석자는 여러 응답을 동시에 순위화할 수 있습니다. 예를 들어, 3개의 순위가 매겨진 응답(A > B > C)은 3개의 순위 쌍을 생성합니다: (A > B), (A > C), (B > C).

<img src="images/fig_02_15.png" width="800">

비교 데이터만 주어진 경우, 모델이 구체적인 점수를 제공하도록 어떻게 훈련할 수 있을까요? 인간에게 적절한 인센티브를 제공하면 원하는 작업을 수행할 수 있듯이, 모델에도 올바른 목표 함수를 제공하면 원하는 작업을 수행할 수 있습니다. 일반적으로 사용되는 함수는 승리 응답과 패배 응답의 점수 차이를 나타냅니다. 목표는 이 차이를 극대화하는 것입니다. 수학적 세부 사항에 관심이 있는 분들을 위해, 여기에 공식이 나와 있습니다:

- $ r_{\theta} $: 훈련 중인 보상 모델, 매개변수화된 $\theta$. 훈련 과정의 목표는 손실(loss)을 최소화하는 $\theta$를 찾는 것입니다.

- **훈련 데이터 형식**
    - $ x $: 프롬프트(prompt)  
    - $ y_w $: 승리 응답(winning response)  
    - $ y_l $: 패배 응답(losing response)  

- $ s_w = r(x, y_w) $: 승리 응답에 대한 보상 모델의 스칼라 점수  
- $ s_l = r(x, y_l) $: 패배 응답에 대한 보상 모델의 스칼라 점수  

- $\sigma$: **시그모이드 함수(Sigmoid function)**  

각 훈련 샘플 $(x, y_w, y_l)$에 대해, 손실 값은 다음과 같이 계산됩니다:

$
\log \left( \sigma \left( r_{\theta}(x, y_w) - r_{\theta}(x, y_l) \right) \right)
$

**목표**: 모든 훈련 샘플에 대해 기대 손실을 최소화하는 $\theta$를 찾습니다.

$
- \mathbb{E} \log \left( \sigma \left( r_{\theta}(x, y_w) - r_{\theta}(x, y_l) \right) \right)
$

보상 모델(RM)은 처음부터 훈련하거나, 사전 훈련된 모델 또는 SFT 모델과 같은 다른 모델 위에 미세 조정할 수 있습니다. 가장 강력한 파운데이션 모델 위에 미세 조정하는 것이 최고의 성능을 제공하는 것으로 보입니다. 일부 사람들은 RM이 파운데이션 모델의 응답을 채점할 수 있으려면 파운데이션 모델만큼 강력해야 한다고 믿습니다. 그러나 다음 장의 평가에서 보게 되듯이, 약한 모델이 더 강력한 모델을 평가할 수 있습니다. 평가(judging)는 생성보다 더 쉬운 작업으로 간주됩니다.

---

**보상 모델을 사용한 미세 조정(Finetuning using the reward model)**

훈련된 보상 모델(RM)을 사용하여, SFT 모델을 추가로 훈련시켜 보상 모델의 점수를 극대화하는 출력 응답을 생성합니다. 이 과정에서 프롬프트는 기존 사용자 프롬프트와 같은 프롬프트 분포에서 무작위로 선택됩니다. 이러한 프롬프트가 모델에 입력되고, 모델의 응답은 보상 모델에 의해 점수가 매겨집니다. 이 훈련 과정은 종종 강화 학습을 사용하며, **RLHF**(Reinforcement Learning from Human Feedback)로 알려져 있습니다. 보다 구체적으로는, OpenAI가 2017년에 발표한 **Proximal Policy Optimization(PPO)** 알고리즘을 사용하여 수행됩니다.

경험적으로, RLHF는 단독 SFT보다 성능을 향상시킵니다. 그러나 이 글을 쓰는 시점에서는 RLHF가 왜 효과가 있는지에 대한 논의가 진행 중입니다. RLHF가 환각(hallucination)을 완화하는지 아니면 악화시키는지도 불분명합니다. 이는 이 장의 "환각(Hallucination)" 섹션에서 논의됩니다. 이 기술은 이론적으로 확립되지 않았고 복잡하기 때문에, 연구가 성숙해짐에 따라 발전하거나 구식이 될 가능성이 있습니다. RLHF에 대해 더 알고 싶다면 **부록 A(Appendix A)**를 참조하세요.

일부 회사는 강화 학습 자체를 생략해도 괜찮다고 판단합니다. 예를 들어, **Stitch Fix**와 **Grab**은 보상 모델만으로도 애플리케이션에 충분히 적합하다고 간주합니다. 그들은 모델이 여러 출력을 생성하도록 한 후, 보상 모델이 높은 점수를 부여한 것을 선택합니다. 이러한 목적으로 사용되는 보상 모델은 **검증기(verifiers)**라고도 불립니다.

SFT와 RLHF는 사전 훈련에 사용된 데이터의 낮은 품질로 인해 발생한 문제를 해결하기 위한 단계라는 점을 상기하세요. 언젠가 더 나은 사전 훈련 데이터나 파운데이션 모델을 훈련시키는 더 나은 방법이 있다면, SFT와 RLHF가 전혀 필요하지 않을 수도 있습니다.

---

## **샘플링(Sampling)**  

모델은 **샘플링(Sampling)**이라고 알려진 과정을 통해 출력을 생성하며, 때로는 **디코딩(Decoding)**이라고도 합니다. 이 섹션에서는 다양한 샘플링 전략과 **샘플링 변수**(예: temperature, top-k, top-p)에 대해 논의합니다. 그런 다음, 모델 성능을 향상시키기 위해 여러 출력을 샘플링하는 방법을 탐구합니다. 또한 샘플링 프로세스가 특정 형식과 제약 조건을 따르는 응답을 생성하도록 수정될 수 있는 방법도 살펴봅니다.

샘플링은 AI의 출력을 확률적으로 만듭니다. 이러한 확률적 특성을 이해하는 것은 AI의 행동(예: 일관성 부족, 환각)을 처리하는 데 중요합니다. 이 섹션은 이러한 확률적 특성이 무엇을 의미하며, 이를 다루는 방법에 대한 심층적인 논의로 끝납니다.

---

### 샘플링 기본(Sampling Fundamentals)

주어진 입력에서, 신경망은 가능한 결과의 확률을 먼저 계산하여 출력을 생성합니다. 분류 모델의 경우, 가능한 결과는 사용할 수 있는 클래스입니다. 예를 들어, 이메일이 스팸인지 아닌지를 분류하도록 모델이 훈련되었다면, 가능한 결과는 "스팸"과 "비스팸" 두 가지뿐입니다. 모델은 이 두 결과 각각의 확률을 계산합니다. 예를 들어, 이메일이 스팸일 확률이 90%, 스팸이 아닐 확률이 10%라고 가정합시다.

다음 토큰을 생성하기 위해, 언어 모델은 어휘(vocabulary)에 있는 모든 토큰에 대한 확률 분포를 먼저 계산합니다. 이는 [그림 2-16](Figure 2-16)과 유사합니다.

<img src="images/fig_02_16.png" width=800>

스팸 이메일 분류 작업에서는 가장 높은 확률을 가진 값을 출력하는 것이 일반적입니다. 이메일이 스팸일 확률이 90%라면, 해당 이메일을 스팸으로 표시할 수 있습니다. 항상 가장 가능성이 높은 결과나 토큰을 선택하는 것을 **탐욕적 샘플링(greedy sampling)**이라고 합니다. 그러나 언어 모델의 경우, 탐욕적 샘플링은 지루한 출력을 생성합니다. 예를 들어, 어떤 질문을 하더라도 항상 가장 흔한 단어로 응답하는 모델을 상상해보세요.

대신, 항상 가장 가능성이 높은 토큰을 선택하는 대신, 모델은 모든 가능한 값에 대한 확률 분포에 따라 다음 토큰을 샘플링할 수 있습니다. 예를 들어, **"My favorite color is ..."**라는 문맥이 주어졌을 때, [그림 2-19](Figure 2-19)에 나타난 대로 "red"가 다음 토큰일 확률이 30%, "green"이 다음 토큰일 확률이 50%라면, "red"는 30%의 확률로 선택되고, "green"은 50%의 확률로 선택됩니다.

다음은 해당 내용을 그대로 번역한 것입니다:

**온도(Temperature), 로그잇(logits), 로그확률(logprobs)**

확률 분포에 따라 다음 토큰을 샘플링하는 한 가지 문제는 모델이 덜 창의적으로 보일 수 있다는 것입니다. 이전 예제에서 "red", "green", "purple"과 같은 색상 단어는 가장 높은 확률을 가질 수 있습니다. 결과적으로 언어 모델의 답변은 다섯 살짜리 아이처럼 들릴 수 있습니다: **"My favorite color is green."** "the"와 같은 단어는 낮은 확률을 가지기 때문에, 모델이 **"My favorite color is the color of a still lake on a spring morning."**과 같은 창의적인 문장을 생성할 가능성은 낮아집니다.

**온도(Temperature)**는 가능한 값의 확률을 재분배하는 기술입니다. 직관적으로, 이는 일반적인 토큰의 확률을 줄이고 드문 토큰의 확률을 증가시킴으로써, 모델이 더 창의적인 응답을 생성할 수 있도록 합니다.

온도가 어떻게 작동하는지 이해하려면, 모델이 확률을 계산하는 방식을 살펴보겠습니다. 입력이 주어지면, 신경망은 이 입력을 처리하여 로그잇 벡터(logit vector)를 출력합니다. 각 **로그잇(logit)**은 하나의 가능한 값에 해당합니다. 언어 모델의 경우, 각 로그잇은 모델의 어휘에 있는 하나의 토큰에 해당합니다. 로그잇 벡터의 크기는 어휘 크기입니다. [그림 2-17](Figure 2-17)은 로그잇 벡터의 시각화를 보여줍니다.

<img src="images/fig_02_17.png" width=800>

로그잇이 클수록 확률이 높아지지만, 로그잇 자체는 확률을 나타내지 않습니다. 로그잇은 1로 합산되지 않으며, 음수가 될 수도 있습니다. 반면 확률은 음수가 될 수 없고, 1로 합산됩니다. 로그잇을 확률로 변환하려면 **소프트맥스(softmax)** 레이어를 사용합니다. 모델의 어휘가 $ N $이고 로그잇 벡터가 $[x_1, x_2, ..., x_N]$이라고 할 때, $ i $번째 토큰의 확률 $ p_i $는 다음과 같이 계산됩니다:

$
p_i = \text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}
$


**온도(Temperature)**는 소프트맥스 변환 전에 로그잇을 조정하기 위해 사용되는 상수입니다. 로그잇은 온도로 나누어집니다. 주어진 온도 $ T $에 대해, $ i $번째 토큰의 조정된 로그잇은 $ x_i / T $입니다. 소프트맥스는 이 조정된 로그잇에 적용됩니다.

간단한 예를 통해 온도가 확률에 미치는 영향을 살펴보겠습니다. 두 가지 가능한 출력 $ A $와 $ B $만 있는 모델이 있다고 가정합시다. 마지막 레이어에서 계산된 로그잇이 $ [1, 3] $인 경우, $ A $의 로그잇은 1이고 $ B $의 로그잇은 3입니다.

- 온도를 사용하지 않을 경우(온도 = 1), 소프트맥스 확률은 $[0.12, 0.88]$입니다. 이 모델은 $ B $를 88%의 확률로 선택합니다.
- 온도가 0.5인 경우, 확률은 $[0.02, 0.98]$입니다. 이 모델은 $ B $를 98%의 확률로 선택합니다.


온도가 높을수록 모델이 가장 명백한 값을 선택할 가능성이 낮아지고(즉, 로그잇 값이 높은 값), 모델의 출력이 더 창의적이지만 덜 일관성이 있게 됩니다. 온도가 낮을수록 모델은 가장 명백한 값을 선택할 가능성이 높아지며, 모델의 출력이 더 일관성이 있지만 잠재적으로 더 지루해질 수 있습니다.

[그림 2-18](Figure 2-18)은 다양한 온도에서 토큰 B의 소프트맥스 확률을 보여줍니다. 온도가 0에 가까워질수록, 모델이 토큰 B를 선택할 확률은 1에 가까워집니다. 예를 들어, 온도가 0.1일 경우, 모델은 거의 항상 B를 출력합니다. 모델 제공자는 일반적으로 온도를 0과 2 사이로 제한합니다. 사용자가 자신의 모델을 소유한 경우, 음이 아닌 모든 온도를 사용할 수 있습니다. **0.7의 온도**는 일반적으로 창의성과 결정성을 균형 있게 유지하는 창의적 사용 사례에서 권장됩니다. 하지만 실험을 통해 자신에게 가장 잘 맞는 온도를 찾아야 합니다.

<img src="images/fig_02_18.png" width=800>

모델의 출력을 더 일관성 있게 하기 위해 온도를 0으로 설정하는 것이 일반적입니다. 기술적으로, 온도는 0이 될 수 없습니다(로그잇을 0으로 나눌 수 없음). 실무적으로, 온도를 0으로 설정하면 모델은 로그잇 조정 및 소프트맥스 계산 없이 가장 큰 로그잇 값을 가진 토큰을 선택합니다.

AI 모델과 작업할 때 일반적인 디버깅 기술은 모델이 주어진 입력에 대해 계산하는 확률을 확인하는 것입니다. 확률이 무작위로 보이는 경우, 모델이 충분히 학습되지 않았을 가능성이 큽니다. OpenAI는 모델이 생성한 확률을 **로그확률(logprobs)**로 반환합니다. **로그확률**은 로그 스케일의 확률로, 신경망의 확률과 작업할 때 선호됩니다. 로그 스케일은 **언더플로우(underflow)** 문제를 줄이는 데 도움이 됩니다. 어휘 크기가 100,000인 언어 모델은 많은 토큰의 확률이 너무 작아 기계에서 표현하기 어려울 수 있습니다. 이러한 작은 숫자는 0으로 반올림될 수 있지만, 로그 스케일은 이 문제를 완화합니다.

[그림 2-19](Figure 2-19)는 로그잇, 확률 및 로그확률이 계산되는 워크플로를 보여줍니다.

<img src="images/fig_02_19.png" width=800>

로그확률은 응용 프로그램을 구축(특히 분류), 평가, 그리고 모델의 작동 방식을 이해하는 데 필수적입니다. 그러나 이 글을 쓰는 시점에서, 많은 모델 제공자는 모델의 로그확률을 공개하지 않거나, 공개하는 경우에도 로그확률 API는 제한적입니다. 제한된 로그확률 API는 기술적 이유보다는, 공개된 로그확률이 다른 사람들이 자신의 모델을 복제하기 쉽게 만들기 때문에 더 가능성이 높습니다.

**Top-k**

**Top-k**는 모델의 응답 다양성을 지나치게 희생하지 않으면서 계산 작업을 줄이는 샘플링 전략입니다. 소프트맥스 레이어가 모든 가능한 값에 대한 확률 분포를 계산하는 데 사용된다는 점을 상기하십시오. 소프트맥스는 두 번의 작업을 필요로 합니다: 첫 번째는 지수 합 계산($\Sigma_j e^{x_j}$), 두 번째는 각 값에 대해 나누기 계산($e^{x_i} / \Sigma_j e^{x_j}$)입니다. 어휘가 큰 언어 모델에서는 이 과정이 계산적으로 비쌉니다.

이 문제를 피하기 위해, 모델이 로그잇을 계산한 후, 우리는 **가장 높은 k개의 로그잇(top-k logits)**을 선택하고, 이들에 대해서만 소프트맥스를 수행합니다. 애플리케이션의 다양성을 얼마나 원하는지에 따라 k 값은 50에서 500 사이가 될 수 있습니다. 이는 모델 어휘 크기보다 훨씬 작습니다. 모델은 이 상위 k개의 값에서 샘플링합니다. k 값이 작을수록 텍스트는 더 예측 가능하지만 덜 흥미롭습니다. 이는 모델이 작고 유사한 단어 집합에만 제한되기 때문입니다.

**Top-p**

**Top-k 샘플링**에서는 고려되는 값의 수가 k로 고정됩니다. 그러나 이 숫자는 상황에 따라 변경되어야 합니다. 예를 들어, 프롬프트가 "Do you like music? Answer with only yes or no."일 경우, 고려해야 할 값의 수는 "yes"와 "no"의 두 가지여야 합니다. 반대로 프롬프트가 **"What’s the meaning of life?"**라면, 고려되는 값의 수는 훨씬 더 커야 합니다.

**Top-p**, 또는 **누클리어스 샘플링(nucleus sampling)**으로도 알려진 방법은 샘플링될 값의 선택을 보다 동적으로 할 수 있게 합니다. Top-p 샘플링에서는 모델이 확률이 높은 다음 값들을 내림차순으로 더하고, 그 합이 p에 도달하면 멈춥니다. 이 누적 확률 내에 있는 값들만 고려됩니다. 언어 모델에서 일반적으로 사용되는 Top-p 값(누클리어스)은 0.9에서 0.95 사이입니다. 예를 들어, Top-p 값이 0.9라면, 모델은 누적 확률이 90%를 초과하는 값의 가장 작은 집합만 고려합니다.

모든 토큰의 확률이 [그림 2-20](Figure 2-20)에 표시된 것처럼 되어 있다고 가정합시다. Top-p가 90%라면, "yes"와 "maybe"만 누적 확률이 90% 이상이기 때문에 고려됩니다. Top-p가 99%라면, "yes", "maybe", 그리고 "no"가 고려됩니다.

<img src="images/fig_02_20.png" width=800>

Top-k와 달리, Top-p는 소프트맥스 계산 부하를 반드시 줄이는 것은 아닙니다. 이 방법의 이점은 각 문맥에 가장 관련성 높은 값들만 초점에 맞추기 때문에 출력이 문맥적으로 더 적합하게 될 수 있다는 점입니다. 이론적으로 Top-p 샘플링은 많은 이점이 없어 보일 수 있지만, 실제로는 효과가 입증되어 인기가 높아지고 있습니다.

**정지 조건(Stopping Condition)**  

자기회귀 언어 모델은 한 번에 하나씩 토큰을 생성하여 토큰 시퀀스를 만듭니다. 긴 출력 시퀀스는 더 많은 시간이 소요되고 더 많은 연산 자원(비용)이 들며, 때로는 사용자에게 불편함을 줄 수 있습니다. 우리는 모델이 시퀀스를 멈추는 조건을 설정할 수 있습니다.

간단한 방법 중 하나는 모델이 고정된 수의 토큰을 생성한 후 멈추도록 요청하는 것입니다. 그러나 이 방법의 단점은 출력이 문장 중간에서 잘릴 가능성이 있다는 점입니다. 또 다른 방법은 **종료 토큰(stop tokens)**을 사용하는 것입니다. 예를 들어, 모델이 `<EOS>`를 만나면 생성을 멈추도록 요청할 수 있습니다. 정지 조건은 지연(latency)과 비용을 줄이는 데 유용합니다.

조기 정지의 단점은 특정 형식의 출력을 생성하려는 경우, 조기 정지가 출력이 잘못된 형식이 되도록 만들 수 있다는 점입니다. 예를 들어, 모델에 JSON을 생성하도록 요청하면, 조기 정지는 닫는 중괄호와 같은 일부 항목이 누락되어 생성된 JSON을 분석하기 어렵게 만들 수 있습니다.

---  

### 테스트 시간 샘플링(Test Time Sampling)

모델의 성능을 개선하는 한 가지 간단한 방법은 여러 출력을 생성한 다음, 가장 좋은 출력이나 가장 일반적인 출력을 선택하는 것입니다. 이 접근법을 **테스트 시간 샘플링(test time sampling)** 또는 **테스트 시간 계산(test time compute)**이라고 합니다. 모델의 응답을 일관되게 유지하려면 모든 샘플링 변수를 고정해야 합니다. 하지만 다양한 출력을 생성하고 그중 최상의 것을 선택하려면, 다양한 출력을 생성하기 위해 샘플링 변수를 다르게 설정하는 것은 피해야 합니다.

최상의 출력을 선택하려면 사용자에게 여러 출력을 보여주고 그들 스스로 가장 적합한 것을 선택하도록 하거나, 최상의 출력을 선택하는 방법을 마련할 수 있습니다. 한 가지 방법은 가장 높은 확률을 가진 출력을 선택하는 것입니다. 언어 모델의 출력은 토큰 시퀀스로 구성되며, 각 토큰은 모델에 의해 확률이 계산됩니다. 출력의 확률은 출력 내 모든 토큰의 확률의 곱입니다.

다음의 토큰 시퀀스를 고려해봅시다: ["I", "love", "food"]. "I"의 확률이 0.2, "love"가 "I"를 따를 확률이 0.1, 그리고 "food"가 "I"와 "love"를 따른다는 조건 하에서의 확률이 0.3이라면, 시퀀스의 확률은 0.2 × 0.1 × 0.3 = 0.006입니다. 수학적으로 이를 다음과 같이 나타낼 수 있습니다:

$
p(\text{I love food}) = p(I) \times p(\text{love }| I) \times p(\text{food }| I, \text{ love})
$

확률을 로그 스케일로 다루는 것이 더 쉽다는 것을 기억하세요. 곱의 로그는 로그의 합과 같으므로, 토큰 시퀀스의 로그확률은 시퀀스 내 모든 토큰의 로그확률의 합입니다:

$
\text{logprob}(\text{I love food}) = \text{logprob}(I) + \text{logprob}(\text{love }| I) + \text{logprob}(\text{food }| I, \text{ love})
$

합을 사용하면 긴 시퀀스는 더 낮은 총 로그확률을 가지는 경향이 있습니다(1의 로그는 0이며, 0과 1 사이의 값의 로그는 음수임). 짧은 시퀀스를 편향하지 않기 위해, 시퀀스의 길이로 합을 나누어 평균 로그확률을 사용합니다. 여러 출력을 샘플링한 후, 평균 로그확률이 가장 높은 출력을 선택합니다. 현재 OpenAI API는 이 방식을 사용합니다. 예를 들어, `best_of` 매개변수를 특정 값(예: 10)으로 설정하여 OpenAI 모델이 10개의 다른 출력 중 평균 로그확률이 가장 높은 출력을 반환하도록 요청할 수 있습니다.

또 다른 방법은 이전 섹션에서 논의된 것처럼 각 출력을 채점하기 위해 보상 모델을 사용하는 것입니다. **Stitch Fix**와 **Grab**은 보상 모델이나 검증기를 통해 높은 점수를 받은 출력을 선택합니다. OpenAI 또한 수학 문제를 해결하도록 검증기를 훈련시켰습니다(Cobbe et al., 2021). 검증기를 사용하는 것이 모델 성능을 크게 향상시킨다는 사실을 발견했습니다. 실제로 검증기를 사용하는 것은 약 30배의 모델 크기 증가와 동일한 성능 향상을 가져왔습니다. 이는 검증기를 사용하는 1억 파라미터 모델이 검증기를 사용하지 않는 30억 파라미터 모델과 동등한 성능을 낼 수 있음을 의미합니다.

같은 실험에서, OpenAI는 더 많은 출력을 샘플링하면 성능이 더 좋아지지만, 특정 지점(400개 출력)까지만 해당된다고 보여주었습니다. 이 지점을 넘어서면 성능이 감소하기 시작합니다. 이는 [그림 2-21](Figure 2-21)에 표시되어 있습니다. 그들은 샘플링된 출력의 수가 증가할수록, 적대적인 출력을 찾을 확률도 증가한다고 가정했습니다. 이는 흥미로운 실험이지만, 실무적으로 하나의 입력에 대해 400개의 다른 출력을 샘플링하는 생산 환경은 없을 것입니다. 비용이 천문학적으로 들 것입니다.

<img src="images/fig_02_21.png" width=800>

응용 프로그램의 필요에 따라 휴리스틱을 선택할 수도 있습니다. 예를 들어, 응용 프로그램이 짧은 응답으로 이점을 얻는 경우 가장 짧은 응답을 선택할 수 있습니다. 응용 프로그램이 자연어를 SQL 쿼리로 변환하도록 설계된 경우, 유효한 SQL 쿼리를 출력할 때까지 출력을 계속 생성하도록 모델에 요청할 수 있습니다.

테스트 시간 샘플링의 특히 흥미로운 응용 사례 중 하나는 지연(latency) 문제를 극복하는 것입니다. 일부 쿼리, 특히 연쇄적 사고(chain-of-thought) 쿼리에서는 모델이 응답을 완료하는 데 오랜 시간이 걸릴 수 있습니다. TIFIN의 AI 책임자인 Kittipat Kampa는 자신의 팀이 모델에 병렬로 여러 응답을 생성하도록 요청한 다음, 사용자에게 먼저 완료된 응답을 보여준다고 말했습니다.

출력 집합 중에서 가장 일반적인 출력을 선택하는 접근 방식은 **자기 일관성(self-consistency)**이라고도 합니다(Wang et al., 2023). 이는 정확한 답변을 기대하는 작업에 특히 유용할 수 있습니다. 예를 들어, 수학 문제를 주어진 경우, 모델이 문제를 여러 번 해결하고 가장 빈번한 답변을 최종 솔루션으로 선택할 수 있습니다. 마찬가지로, 선택형 문제에서 모델은 가장 빈번한 출력 옵션을 선택할 수 있습니다. 이것이 Google이 MMLU에서 Gemini 모델을 평가할 때 했던 방식입니다. MMLU는 선택형 문제의 벤치마크로, 각 질문에 대해 32개의 출력을 샘플링했습니다. 이는 Gemini가 이 벤치마크에서 높은 점수를 얻는 데 도움이 되었지만, 질문당 하나의 출력만 생성하는 다른 모델보다 Gemini가 더 나은지 여부는 불분명합니다.

모델의 출력이 입력에 약간의 변화가 있어도 거의 동일하게 유지된다면, 모델은 견고하다고 간주됩니다. 모델이 덜 견고할수록, 여러 출력을 샘플링함으로써 얻을 수 있는 이점이 커집니다. 한 프로젝트에서 우리는 AI를 사용하여 제품 이미지에서 특정 정보를 추출했습니다. 동일한 이미지에서 모델이 절반의 경우만 정보를 정확히 읽을 수 있다는 것을 발견했습니다. 나머지 절반에서는 이미지가 너무 흐리거나 텍스트가 너무 작아 읽을 수 없다고 모델이 응답했습니다. 각 이미지에 대해 모델이 정보를 추출할 때까지 최대 세 번 쿼리해야 했습니다.

여러 출력을 샘플링하면 일반적으로 모델 성능 향상을 기대할 수 있지만, 비용이 많이 듭니다. 평균적으로 두 개의 출력을 생성하는 비용은 하나의 출력을 생성하는 비용의 약 두 배에 해당합니다.

---

### 구조화된 출력(Structured Outputs)

종종, 실제 환경에서는 특정 형식을 따르는 텍스트를 생성해야 하는 모델이 필요합니다. 구조화된 출력은 다음 두 가지 시나리오에서 중요합니다:

1. **구조화된 출력을 요구하는 작업**  
   이 시나리오에서 가장 일반적인 작업 범주는 **의미 구문 분석(semantic parsing)**입니다. 의미 구문 분석은 자연어를 구조화된, 기계가 읽을 수 있는 형식으로 변환하는 작업을 포함합니다. Text-to-SQL은 의미 구문 분석의 예로, 출력이 유효한 SQL 쿼리여야 합니다. 의미 구문 분석을 통해 사용자는 영어와 같은 자연어를 사용하여 API와 상호작용할 수 있습니다. 예를 들어, Text-to-PostgreSQL은 사용자가 **"지난 6개월 동안의 평균 월 매출은?"**과 같은 영어 쿼리를 사용하여 Postgres 데이터베이스를 쿼리할 수 있도록 합니다. 이는 PostgreSQL 쿼리를 직접 작성하지 않고도 가능합니다.

   다음은 GPT-4를 사용하여 텍스트를 정규식(regex)으로 변환하는 프롬프트 예제입니다. 출력은 실제로 GPT-4에 의해 생성된 출력입니다.

   **시스템 프롬프트**  
   주어진 항목에 대해 해당 항목이 작성될 수 있는 모든 방식을 나타내는 정규식을 작성하세요. 정규식만 반환하세요.

   **예시**  
   US phone number → \+?\d?\s?(\()?(\d{3})(?(1)\))[-.\s]?(\d{3})[-.\s]?(\d{4})

   **사용자 프롬프트**  
   Email address  

   **GPT-4의 출력**  
   [a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}

   **사용자 프롬프트**  
   Dates  

   **GPT-4의 출력**  
   (?:\d{1,2}[/\-\.])(?:\d{1,2}[/\-\.])?\d{2,4}

   이 시나리오에서의 다른 작업 범주로는 출력이 유효한 클래스여야 하는 분류(classification)가 포함됩니다.

2. **다운스트림 애플리케이션에서 사용되는 출력이 필요한 작업**  
   이 시나리오에서는 작업 자체가 구조화된 출력을 필요로 하지 않지만, 다른 애플리케이션에서 출력을 사용하기 때문에 이 출력을 애플리케이션이 구문 분석할 수 있어야 합니다.

   예를 들어, AI 모델을 사용하여 이메일을 작성하는 경우, 이메일 자체는 구조화될 필요가 없습니다. 그러나 이 이메일을 사용하는 다운스트림 애플리케이션은 JSON 문서 내의 특정 형식(예: `{"title": [TITLE], "body": [EMAIL BODY]}`)이 필요할 수 있습니다.

   이는 모델의 출력이 특정 함수로 입력되는 에이전트 워크플로에서 특히 중요합니다. 이는 **5장**에서 논의됩니다.

여러 모델의 구조화된 출력을 지원하는 도구에는 **guidance**, **outlines**, **instructor**, **llama.cpp**가 포함됩니다. **LLaMA** 모델 패밀리를 위한 제약 샘플링을 지원합니다. 각 모델 제공자는 구조화된 출력을 생성하는 모델의 능력을 개선하기 위해 자체 기술을 사용할 수도 있습니다.

**OpenAI**는 텍스트 생성 API에 **JSON 모드**를 도입한 최초의 모델 제공자였습니다. 이 JSON 모드는 출력이 유효한 JSON임을 보장하지만, JSON 객체의 내용에 대한 보장은 하지 않습니다. 생성된 JSON이 너무 빨리 중단되면(예: 최대 출력 토큰 길이에 도달했을 때), 그렇지 않으면 유효한 JSON도 구문 분석이 불가능할 수 있습니다. 최대 토큰 길이가 너무 짧게 설정되면 출력 JSON이 잘릴 수 있으며, 따라서 구문 분석이 불가능해집니다. 반대로 너무 길게 설정되면 모델의 응답이 너무 느리고 비싸질 수 있습니다.

[그림 2-22](Figure 2-22)는 옵션 집합과 정규식에 제한된 출력을 생성하기 위해 가이던스를 사용하는 두 가지 예를 보여줍니다.

<img src="images/fig_02_22.png" width=800>

모델이 AI 스택의 다른 레이어(프롬프트 작성, 후처리, 테스트 시간 샘플링, 제약 샘플링, 파인튜닝)에서 구조화된 출력을 생성하도록 안내할 수 있습니다. 처음 세 가지 방법은 "응급처치"에 가깝습니다. 모델이 이미 구조화된 출력을 생성하는 데 꽤 능숙하고 약간의 조정만 필요한 경우에 가장 잘 작동합니다. 더 심층적인 작업이 필요하다면, 제약 샘플링과 파인튜닝이 필요합니다.

테스트 시간 샘플링은 이전 섹션에서 이미 논의되었습니다. 예상 형식에 맞는 출력이 생성될 때까지 출력을 계속 생성하는 방법입니다. 이번 섹션은 나머지 네 가지 접근 방식에 초점을 맞춥니다.

---

**프롬프트 작성(Prompting)**

프롬프트 작성은 구조화된 출력을 위한 첫 번째 작업입니다. 모델에 특정 스키마를 따르는 유효한 **JSON** 또는 **YAML** 출력을 생성하도록 지시할 수 있습니다. 그러나 모델이 이 지시를 따를 수 있는지는 모델의 지시-따르기 능력(4장에서 논의됨)과 지시의 명확성(5장에서 논의됨)에 달려 있습니다. 모델이 지시를 따르는 능력이 점점 더 좋아지고는 있지만, 일반적인 기초 모델이 항상 주어진 스키마를 따라 유효한 JSON 객체를 출력할 것이라고 보장할 수는 없습니다. 이 글을 작성하는 시점에서, 애플리케이션 및 모델에 따라 올바르게 생성된 JSON 객체의 비율은 0%에서 높은 90%에 이릅니다. 유효하지 않은 출력 몇 퍼센트도 많은 애플리케이션에서는 여전히 용납되지 않을 수 있습니다.

유효한 출력의 비율을 높이기 위해, 일부 사용자는 원래 프롬프트의 출력의 유효성을 검증하거나 수정하는 또 다른 프롬프트를 사용합니다. 이는 3장에서 논의된 "판사 역할을 하는 AI" 접근 방식의 예입니다. 이 접근 방식은 각 출력에 대해 최소 두 개의 모델 쿼리가 필요합니다: 하나는 출력을 생성하고, 다른 하나는 이를 검증합니다. 추가 검증 레이어가 출력의 유효성을 상당히 향상시킬 수 있지만, 이 추가 검증 쿼리로 인해 발생하는 추가 비용과 지연으로 인해 일부 사용자에게는 이 접근 방식이 너무 비쌀 수 있습니다.

---  

**후처리(Post-processing)**

후처리는 간단하고 저렴하면서도 놀라울 정도로 효과적으로 작동할 수 있습니다. 제가 강의할 때, 학생들이 매우 유사한 실수를 자주 한다는 것을 발견했습니다. 기본 모델로 작업을 시작했을 때도 동일한 현상을 관찰했습니다. 모델은 쿼리 간에 유사한 실수를 반복하는 경향이 있습니다. 이는 모델이 자주 저지르는 공통적인 실수를 발견하면, 이를 수정하기 위해 스크립트를 작성할 수 있음을 의미합니다. 예를 들어, 생성된 JSON 객체가 닫는 괄호를 놓쳤다면, 수동으로 해당 괄호를 추가할 수 있습니다. LinkedIn의 방어적 YAML 파서는 정확한 YAML 출력의 비율을 90%에서 99.99%로 증가시켰습니다(Bottaro et al., 2020).

>**팁(TIP)**  
>
>JSON과 YAML은 일반적인 텍스트 형식입니다. LinkedIn은 그들의 기본 모델인 GPT-4가 둘 다 처리할 수 있다는 사실을 발견했지만, 출력 토큰이 더 적게 필요하고 덜 장황하기 때문에 YAML을 출력 형식으로 선택했습니다(Bottaro et al., 2020).

후처리는 실수가 쉽게 수정될 수 있는 경우에만 효과적입니다. 이는 일반적으로 모델의 출력이 대부분 올바르게 포맷되고, 간헐적으로 작은 오류가 발생하는 경우에 해당됩니다.

**제약 샘플링(Constrained Sampling)**

**제약 샘플링(Constraint sampling)**은 텍스트 생성을 특정 제약에 맞추기 위해 사용하는 기술입니다. 이는 구조화된 출력 도구에서 일반적으로 따르는 접근 방식입니다.

고수준에서 토큰을 생성하려면, 모델이 제약을 충족하는 값 중에서 샘플링을 수행합니다. 기억하십시오, 토큰을 생성하려면 모델이 로그잇(logit) 벡터를 출력하며, 각 로그잇은 하나의 가능한 토큰에 해당합니다. 제약 샘플링은 이 로그잇 벡터를 필터링하여 제약 조건을 충족하는 토큰만 유지합니다. 그런 다음 이러한 유효한 토큰 중에서 샘플링합니다. 이 과정은 [그림 2-24](Figure 2-24)에 설명되어 있습니다.

[그림 2-24](Figure 2-24)의 예에서, 제약 조건은 필터링하기 쉽습니다. 하지만 대부분의 경우에는 그렇지 않습니다. 각 단계에서 허용되는 것과 허용되지 않는 것을 지정하는 문법(grammar)이 필요합니다. 예를 들어, JSON 문법은 `{` 뒤에 또 다른 `{`가 올 수 없음을 명시하며, 이는 문자열의 일부인 경우를 제외하고 적용됩니다(예: `{"key": "{{string}}"}`).

<img src="images/fig_02_23.png" width=800>

이 문법을 구축하고 샘플링 프로세스에 통합하는 것은 간단하지 않습니다. JSON, YAML, 정규식(regex), CSV 등 모든 출력 형식에 대해 별도의 문법이 필요합니다. 이는 제약 샘플링을 덜 일반화 가능하게 만듭니다. 이러한 기술은 팀이 문법을 작성할 수 있는 형식 또는 구조화된 출력 도구가 지원하는 형식으로 제한됩니다. 일부는 제약 샘플링에 필요한 자원이 지시를 따르는 능력을 향상시키기 위해 모델을 훈련하거나 파인튜닝하는 데 더 잘 사용될 수 있다고 믿습니다.

---  

**파인튜닝(Finetuning)**

파인튜닝은 원하는 스타일과 형식으로 출력을 생성하도록 모델을 조정하는 가장 효과적이고 일반적인 접근 방식입니다. 이는 예상되는 모든 형식과 작동할 수 있습니다. 모델의 아키텍처를 변경하지 않고도, 또는 변경하여 파인튜닝을 수행할 수 있습니다. 예를 들어, 원하는 출력 형식을 가진 예제를 사용하여 모델을 파인튜닝할 수 있습니다. 이렇게 해도 모델이 항상 예상되는 형식을 출력한다고 보장할 수는 없지만, 프롬프트 작성보다 훨씬 신뢰할 수 있습니다. 또한, 원하는 형식의 지시와 예제를 프롬프트에 포함할 필요가 없다는 점에서 추론 비용을 줄이는 추가적인 이점이 있습니다.

특정 작업의 경우, 모델의 아키텍처를 수정하여 파인튜닝을 통해 출력 형식을 보장할 수 있습니다. 예를 들어, 분류 작업에서는 기본 모델의 아키텍처에 분류기 헤드를 추가하여 모델이 사전에 지정된 클래스 중 하나만 출력하도록 보장할 수 있습니다. 이 아키텍처는 **[그림 2-24](Figure 2-24)**와 유사합니다. 이 접근 방식은 **특징 기반 전이 학습(feature-based transfer learning)**이라고도 하며, 이는 **6장**에서 더 자세히 논의됩니다.

<img src="images/fig_02_24.png" width=800>

파인튜닝 동안, 전체 아키텍처를 처음부터 끝까지 재훈련하거나 이 분류기 헤드만 재훈련할 수 있습니다. 처음부터 끝까지의 훈련은 더 많은 자원을 필요로 하지만, 더 나은 성능을 약속합니다.

샘플링 및 파인튜닝 기술은 모델 자체가 이를 수행할 능력이 없다는 가정 때문에 필요합니다. 모델이 더 강력해짐에 따라, 지시를 따르는 능력이 더 나아질 것으로 기대할 수 있습니다. 미래에는 최소한의 프롬프트로 필요한 출력을 정확히 생성할 수 있는 모델을 얻기가 더 쉬워질 것이며, 이러한 기술의 중요성은 줄어들 것으로 예상합니다.

---  

### AI의 확률적 특성(The Probabilistic Nature of AI)

AI 모델이 응답을 샘플링하는 방식은 이를 **확률적(probabilistic)**으로 만듭니다. 확률적이란 무엇을 의미하는지 예제를 통해 살펴봅시다. 세계 최고의 요리는 무엇인지 알고 싶다고 상상해보세요. 친구에게 이 질문을 두 번 물으면, 몇 분 간격으로도 답변은 동일할 것입니다. 하지만 동일한 질문을 AI 모델에 두 번 물으면, 답변이 달라질 수 있습니다. 예를 들어, AI 모델이 베트남 요리가 세계 최고의 요리일 확률을 70%, 이탈리아 요리가 그럴 확률을 30%라고 생각한다면, AI는 **"베트남 요리"**를 70%의 확률로 답하고, **"이탈리아 요리"**를 30%의 확률로 답할 것입니다. 확률적의 반대는 **결정론적(deterministic)**입니다. 이는 결과가 임의의 변동 없이 결정될 수 있을 때를 말합니다.

이러한 확률적 특성은 **불일치(inconsistency)**와 **환각(hallucination)**을 유발할 수 있습니다.  
**불일치**는 모델이 동일하거나 약간 다른 프롬프트에 대해 매우 다른 응답을 생성할 때 발생합니다.  
**환각**은 모델이 데이터에 기반하지 않은 응답을 생성할 때 발생합니다. 예를 들어, 인터넷 어딘가에 모든 미국 대통령이 외계인이라는 에세이가 있다고 가정합시다. 모델이 이 데이터를 학습한 후, 확률적으로 현재 미국 대통령이 외계인이라고 출력한다면, 이는 사실이 아닌 것을 만들어낸 것입니다. 이는 미국 대통령이 외계인이라고 믿지 않는 사람의 관점에서 보면, 모델이 이를 지어내고 있는 것입니다.

기초 모델은 일반적으로 대량의 데이터를 사용하여 훈련됩니다. 이러한 데이터는 수많은 사람들의 의견 집합으로, 문자 그대로 무한한 가능성을 포함합니다. 확률이 0이 아닌 한, 얼마나 터무니없는 것이든지 간에, AI에 의해 생성될 수 있습니다.  

이 특성은 AI 애플리케이션을 구축하는 것을 흥미롭고 도전적으로 만듭니다. 많은 AI 엔지니어링 노력은 이러한 확률적 특성을 활용하고 완화하는 것을 목표로 합니다. 이 책에서 보게 될 것처럼 말입니다.

이러한 확률적 특성은 창의적인 작업에서 AI를 훌륭한 도구로 만듭니다. 창의성이란 공통적인 경로를 넘어 새로운 것을 탐험할 수 있는 능력 아닐까요? AI는 창의적인 전문가들에게 훌륭한 조력자가 될 수 있습니다. AI는 무한한 아이디어를 브레인스토밍하고 이전에 본 적 없는 디자인을 생성할 수 있습니다. 하지만, 이 확률적 특성은 다른 모든 작업에서는 문제를 일으킬 수 있습니다.

---

**불일치(Inconsistency)**

**불일치**는 모델이 동일한 프롬프트 또는 약간 다른 프롬프트에 대해 매우 다른 응답을 생성할 때 발생합니다. 이러한 불일치는 사용자 경험을 혼란스럽게 만들 수 있습니다. 인간 대 인간의 커뮤니케이션에서 우리는 일정 수준의 일관성을 기대합니다. 보험 회사가 웹사이트를 확인할 때마다 다른 견적을 제공한다고 상상해보세요. 이는 신뢰를 손상시킬 수 있습니다.  

[그림 2-25](Figure 2-25)는 동일한 프롬프트를 두 번 실행했을 때 ChatGPT가 에세이를 채점하려고 시도했던 예를 보여줍니다. 동일한 프롬프트에서 두 번 실행했을 때 점수는 7/10과 4/10으로 다르게 나왔습니다.

<img src="images/fig_02_25.png" width=800>

이러한 불일치가 나타나는 두 가지 시나리오가 있습니다:  
1. **같은 입력, 다른 출력(Same input, different outputs):** 동일한 프롬프트를 모델에 제공하면 두 가지 매우 다른 응답을 생성합니다.  
2. **약간 다른 입력, 크게 다른 출력(Slightly different input, drastically different outputs):** 프롬프트를 약간 다르게 제공했을 때, 예를 들어 대문자를 실수로 사용한 경우, 매우 다른 결과를 생성합니다.

첫 번째 시나리오에서는, 모델의 출력 생성 변수를 고정하여 불일치를 완화할 수 있습니다. 앞서 논의된 온도(temperature), top-k, top-p 값을 고정할 수 있습니다. 또한 **시드(seed)** 변수를 고정할 수도 있습니다. 시드는 다음 토큰 샘플링에 사용되는 난수 생성기의 시작점을 생각하면 됩니다.

이 모든 변수를 고정하더라도, 모델이 항상 100% 일관성을 유지할 것이라고 보장할 수는 없습니다. 모델이 실행되는 하드웨어는 출력 생성에 영향을 줄 수 있으며, 서로 다른 기계는 동일한 명령을 실행하는 방식과 숫자를 처리하는 범위가 다를 수 있습니다. 만약 모델을 자체적으로 호스팅한다면, 사용하는 하드웨어를 어느 정도 제어할 수 있습니다. 하지만 OpenAI나 Google과 같은 제공업체가 호스팅하는 모델을 사용하는 경우, 이들 제공업체가 사용하는 시스템을 제어할 수 없습니다. OpenAI의 **`system_fingerprint`**와 같은 변수는 해당 모델이 실행되는 시스템이 변경되었는지 여부를 확인할 수 있도록 도와줍니다.

출력 생성 설정을 고정하는 것은 좋은 관행이지만, 시스템에 대한 신뢰를 불러일으키지는 않습니다. 한 교사가 특정 방에서만 일관된 점수를 준다고 상상해 보세요. 만약 그 교사가 다른 방에 앉아 있다면, 그 점수는 크게 달라질 것입니다.

두 번째 시나리오는 더 도전적입니다. 모델의 출력 생성 변수를 고정하는 것은 여전히 좋은 방법이지만, 이는 모델이 서로 다른 입력에 대해 동일한 출력을 생성하도록 강제하지는 않습니다. 다만, 신중하게 작성된 프롬프트를 사용하면 모델이 원하는 응답에 더 가깝게 출력하도록 만들 수 있습니다. **5장**에서는 프롬프트 엔지니어링에 대해 논의합니다.

**환각(Hallucination)**

사실성에 의존하는 작업에서 환각은 치명적일 수 있습니다. 만약 AI에게 백신의 장단점을 설명하도록 요청했는데, AI가 비과학적인 정보를 제공한다면 이는 큰 문제가 됩니다. 2023년 6월, 한 로펌이 허위 법률 연구를 제출했다는 이유로 벌금을 물었습니다. 이 로펌은 ChatGPT를 사용해 사례를 준비했으나, ChatGPT가 환각을 일으킬 가능성을 인지하지 못했습니다.

이 책은 기초 모델의 문맥에서 환각을 다루지만, 환각은 기초 모델이나 트랜스포머 아키텍처가 도입되기 전부터 생성 모델에서 흔히 발생하던 현상이었습니다. 2016년까지도 텍스트 생성 문맥에서 환각이 언급되었으며, 자연어 생성(NLG)에서 환각을 탐지하고 측정하는 것은 꾸준히 이어져 온 주제입니다(참조: Goyal et al., 2016; Lee et al., 2018; Nie et al., 2019; Zhou et al., 2020).

만약 샘플링 과정의 무작위성이 불일치의 원인이라면, 환각의 원인은 더욱 미묘합니다. 샘플링 과정만으로는 환각을 충분히 설명할 수 없습니다. 모델은 모든 가능한 옵션에서 출력을 샘플링합니다. 그렇다면 어떻게 전혀 보지 못한 것이 가능한 옵션이 될 수 있을까요? 모델은 이전에 본 적이 없다고 믿어지는 것을 출력할 수 있습니다. 하지만 훈련 데이터를 모두 조사해 특정 아이디어가 언급되었는지 확인하는 것은 불가능하므로, 이 문제를 확신할 수 없습니다. 너무 복잡한 것을 구성하는 우리의 능력은 축복이자 저주입니다.

환각의 원인을 이해하지 않고는 이를 제거하기가 어렵습니다. 현재 언어 모델이 왜 환각을 일으키는지에 대한 두 가지 가설이 있습니다.

첫 번째 가설은 2021년 **Pedro A. Ortega et al. at DeepMind**가 제시한 것으로, 언어 모델은 외부 데이터(훈련 데이터 또는 사용자가 제공한 문맥)와 모델이 생성한 데이터를 구별하지 못하기 때문에 환각을 일으킨다는 것입니다. 언어 모델은 사용자 제공 문맥과 모델이 이전에 생성한 토큰으로 구성된 기존 시퀀스를 바탕으로 다음 토큰을 생성합니다.

예를 들어, "Who’s Chip Huyen?"이라는 프롬프트를 모델에 입력했다고 가정해 봅시다. 모델이 생성한 첫 번째 문장은 "Chip Huyen is an architect."입니다. 이 다음에 모델은 "Who’s Chip Huyen? Chip Huyen is an architect."라는 시퀀스를 바탕으로 다음 토큰을 생성합니다. 오늘날의 모델은 사용자가 제공한 프롬프트와 이전에 생성한 텍스트를 구별하지 못합니다. 모델은 "Chip Huyen is an architect."를 외부 사실로 취급하며, 이 다소 비정상적인 시퀀스를 시작으로 모델은 더욱 엉뚱하고 터무니없는 사실들을 생성하게 됩니다. Ortega와 다른 저자들은 이러한 환각을 **자기기만(self-delusion)**의 형태라고 불렀습니다.

**Figure 2-26**은 LLaVA-v1.5-7B 모델에서 발생한 자기기만의 예를 보여줍니다. 모델이 이미지에 있는 제품 라벨의 재료를 식별하도록 요청받았고, 이 제품은 샴푸였습니다. 그러나 응답에서 모델은 이 제품이 우유병이라고 확신하며, 추출된 재료 목록에 우유를 포함시킵니다.

<img src="images/fig_02_26.png" width=800>

이들은 환각을 두 가지 기술로 완화할 수 있다고 이론화했습니다. 첫 번째 기술은 강화 학습에서 비롯된 것으로, 모델이 사용자가 제공한 프롬프트(강화 학습에서 '세계 관찰(observations about the world)'이라 부름)와 모델이 생성한 토큰(모델의 '행동(actions)'이라 부름)을 구별할 수 있도록 모델을 훈련하는 것입니다. 두 번째 기술은 지도 학습을 기반으로 하며, 사실적 신호와 반사실적 신호가 훈련 데이터에 포함됩니다.

두 번째 가설은 모델의 내부 지식과 레이블러의 내부 지식 간의 불일치가 환각의 원인이라는 것입니다. 이 관점은 OpenAI 직원인 **Leo Gao**가 처음 제기했습니다. SFT(Supervised Fine-Tuning) 동안, 모델은 레이블러가 작성한 응답을 모방하도록 훈련됩니다. 만약 이 응답이 모델이 가지고 있지 않은 지식을 포함한다면, 우리는 모델에게 환각을 가르치고 있는 셈이 됩니다. 이론적으로, 레이블러가 응답에 사용하는 지식을 포함시켜 모델이 응답이 만들어진 것이 아님을 알 수 있도록 하면 모델을 훈련할 수 있을 것입니다. 하지만 실제로는 이는 불가능합니다.

2023년 4월, OpenAI 공동 창립자인 **John Schulman**은 **UC Berkeley 강연**에서 같은 견해를 표현했습니다. Schulman은 "대규모 언어 모델(LLMs)은 자신이 아는 것을 안다"고 믿으며, 이는 자체적으로 매우 큰 주장입니다. 이 믿음이 사실이라면, 환각은 모델이 알고 있는 정보만을 기반으로 답변을 하도록 강제함으로써 해결될 수 있습니다. 그는 두 가지 해결책을 제안했습니다. 하나는 각 응답의 출처를 검색하도록 모델에게 요청하는 검증이며, 또 다른 하나는 강화 학습을 사용하는 것입니다. 보상 모델은 단지 비교만을 사용해 훈련됩니다. 예를 들어, 응답 A가 응답 B보다 더 좋다는 것은 설명 없이 이루어집니다. Schulman은 더 나은 보상 함수가 모델이 근거 없는 정보를 만들어내는 것을 처벌함으로써 환각을 완화할 수 있다고 주장했습니다.

같은 강연에서, Schulman은 OpenAI가 RLHF(보상을 통한 강화 학습)가 환각을 줄이는 데 도움을 준다는 것을 발견했다고 언급했습니다. 그러나 InstructGPT 논문은 RLHF가 환각을 오히려 악화시켰음을 보여줍니다(참조: **Figure 2-27**). RLHF가 InstructGPT에서의 환각을 악화시켰음에도 불구하고, 다른 측면에서는 개선되었으며, 전반적으로 인간 레이블러들은 SFT 단독 모델보다 RLHF 모델을 선호했습니다.

<img src="images/fig_02_27.png" width=800>

기초 모델이 자신이 아는 것을 안다는 가정에 기반하여, 일부 사람들은 다음과 같은 프롬프트를 추가하여 환각을 줄이려고 합니다: "가능한 한 진실되게 대답하고, 답이 확실하지 않다면 '죄송합니다, 알지 못합니다.'라고 말하십시오." 또한, 간결한 응답을 요청하는 것도 환각을 줄이는 데 도움이 되는 것으로 보입니다. 모델이 생성해야 하는 토큰이 적을수록, 모델이 새로운 내용을 만들어낼 가능성도 줄어듭니다.

논의된 두 가지 가설은 서로를 보완합니다. 자기기만(self-delusion) 가설은 자기지도 학습이 환각을 유발하는 방식에 초점을 맞추는 반면, 불일치된 내부 지식 가설은 지도 학습이 환각을 유발하는 방식에 초점을 맞춥니다.

우리가 환각을 완전히 막을 수 없다면, 최소한 모델이 환각된 응답을 사용자에게 제공하지 않도록 이를 감지할 수 있어야 합니다. 그러나 환각을 감지하는 일은 간단하지 않습니다. 다른 사람이 거짓말을 하거나 사실을 날조하는 것을 감지하는 일이 얼마나 어려운지 생각해보십시오. 4장에서 환각을 감지하고 측정하는 방법을 논의할 것입니다.

---

# 요약

이번 장에서는 기초 모델(foundation model)을 구축할 때 고려해야 할 기본 사항들을 다루었습니다. 대부분의 사람들은 처음부터 모델을 훈련시키는 대신 기성의 기초 모델을 사용하는 경우가 많으므로, 훈련의 세부 사항은 생략하고 어떤 모델을 선택하고 사용하는지 결정하는 데 도움을 줄 수 있는 요소들에 중점을 두었습니다.

가장 먼저 알아야 할 것은 모델이 어떤 데이터를 바탕으로 훈련되었는지입니다. 대규모 모델은 많은 양의 훈련 데이터를 필요로 하며, 이는 생산하기에 비용이 많이 들고 시간이 오래 걸립니다. 따라서 모델 제공업체들은 대개 대량으로 사용할 수 있는 데이터를 활용합니다. 이는 다양한 작업에는 적합할 수 있지만, 특정 작업에서 뛰어난 성능을 보장하지는 못합니다. 저희는 특히 저자원 언어와 같은 특정 언어를 위해 개발된 모델, 그리고 의료나 법률과 같은 특정 도메인에 맞춰 개발된 모델을 살펴보았습니다.

데이터를 확보한 후에는 모델 훈련을 시작할 수 있습니다. 기초 모델 훈련은 일반적으로 두 단계로 구성됩니다: 사전 훈련(pre-training)과 후속 훈련(post-training).

사전 훈련 섹션에서는 모델 아키텍처와 모델 크기와 같은 구성 요소를 살펴보았습니다. 모델의 규모는 매개변수(parameter)의 수, 훈련 데이터의 토큰 수, 훈련에 필요한 연산량(FLOPs)으로 측정할 수 있습니다. 모델 크기와 데이터 크기는 훈련에 필요한 연산량에 영향을 미치는 두 가지 측면입니다. 스케일링 법칙(scaling law)은 주어진 컴퓨팅 예산에 따라 최적의 매개변수 수와 토큰 수를 결정하는 데 도움을 줍니다. 또한 스케일링의 병목 현상도 살펴보았습니다. 지금까지는 모델의 크기를 키우는 것이 일반적으로 성능을 향상시켰지만, 이것이 언제까지 계속될지는 알 수 없습니다.

사전 훈련 중 낮은 품질의 훈련 데이터와 자기 지도 학습(self-supervision)으로 인해, 결과적으로 사용자가 원하는 것과 일치하지 않는 출력을 생성하는 모델이 만들어질 수 있습니다. 이는 후속 훈련을 통해 해결됩니다. 후속 훈련은 크게 두 단계로 구성됩니다: 감독된 미세 조정(supervised fine-tuning)과 인간의 선호에 맞춘 조정(alignment)입니다. 인간의 선호는 다양하며, 단일 수학 공식으로 캡처할 수 없으므로 기존 솔루션은 완벽하지 않습니다.

이번 장에서는 제가 가장 좋아하는 주제 중 하나인 샘플링(sampling)도 다뤘습니다. 샘플링은 모델이 출력 토큰을 생성하는 과정입니다. 샘플링의 확률적 성격은 ChatGPT와 Gemini 같은 모델이 창의적인 작업에 뛰어나고 재미있는 이유입니다. 그러나 이러한 확률적 성격은 불일치와 환각(hallucination)도 유발합니다.

모델이 어떻게 훈련되었고 어떤 데이터를 기반으로 했는지를 이해하는 것은 모델이 어떤 작업에 적합한지 이해하는 데 중요합니다. 그러나 모델 제공업체들이 자신들을 공공 및 경쟁사의 비판으로부터 보호하기 위해 점점 더 비밀스러워짐에 따라, 응용 프로그램 개발자가 모델 환경을 탐색하기 어려워지고 있습니다.

경험 많은 AI 엔지니어들은 AI의 확률적 성격을 수용하고 워크플로우를 이에 맞춰 조정했다고 말합니다. 이는 결정론적 프로그램을 개발하는 접근 방식과는 다르지만, 익숙해지는 데 시간이 걸리더라도 불가능하지는 않습니다. 이 책의 나머지 부분에서는 적어도 결정론적이지는 않더라도, 체계적인 AI 엔지니어링 방법을 만드는 방법을 다룰 것입니다.

---

1. **사전 훈련**(pre-training)과 **후속 훈련**(post-training)이라는 용어가 상상력이 부족하다고 느껴진다면, 혼자가 아닙니다. AI 연구 커뮤니티는 많은 것을 잘하지만, 이름 짓는 것은 그 중 하나가 아닙니다. 1장에서 이미 "대규모 언어 모델"(large language models)이라는 용어가 얼마나 모호한지 언급했습니다. 그리고 저는 "X가 전부다"라는 제목의 논문을 그만 발표했으면 좋겠다고 생각합니다.

2. 잘못 정렬된(misaligned) 모델이 더 나을 수 있는 상황도 있습니다. 예를 들어, 사람들이 AI를 사용해 잘못된 정보를 퍼뜨릴 위험을 평가하려고 할 때, 가능한 한 가짜 뉴스를 잘 만들어내는 모델을 구축해 AI의 설득력을 확인해볼 수 있습니다.

3. 6장에서는 **멀티태스크 미세 조정**(multi-task finetuning)을 더 자세히 다룰 것입니다.

4. 그리고 **놀라운 보상 패키지**를 제공합니다.

5. **"Neural Machine Translation by Jointly Learning to Align and Translate"** (Bahdanau et al., 2015)

6. **"RWKV: Reinventing RNNs for the Transformer Era"** (Peng et al., 2023)

7. 재미있는 사실: Ilya Sutskever(OpenAI 공동 창립자)는 seq2seq 논문의 제1저자이자 AlexNet 논문의 제2저자입니다.

8. 참고 사항: **"Transformer는 처음에 Google에서 TPU에서 빠르게 실행되도록 설계되었으며, 이후 GPU에 최적화되었습니다."**

9. 실제 필요한 메모리는 **소프트웨어 비효율성 및 ML 모델 실행과 관련된 오버헤드**로 인해 더 높습니다.

10. **"Outrageously Large Neural Networks: The Sparsely-Gated Mixture-of-Experts Layer"**

11. 한 권의 책이 약 50,000단어 또는 67,000 토큰으로 구성된다고 가정합니다.

12. 작성 당시, 대규모 모델은 일반적으로 데이터 세트의 한 에포크만 학습합니다. 

13. **교차 엔트로피 손실**(Cross entropy loss)은 3장에서 다룹니다.

14. **Nats**는 교차 엔트로피 손실을 측정하는 단위입니다.

15. 친구가 이 비유를 사용했습니다: **"사전 훈련된 모델은 웹 페이지처럼 말한다, 인간이 아니라."**

16. 잘못 정렬된(misaligned) 모델이 더 나을 수 있는 상황도 있습니다. 예를 들어, 사람들이 AI를 사용해 잘못된 정보를 퍼뜨릴 위험을 평가하려고 할 때, 가능한 한 가짜 뉴스를 잘 만들어내는 모델을 구축해 AI의 설득력을 확인해볼 수 있습니다.

17. **"Chatbot Arena: An Open Platform for Evaluating LLMs by Human Preference"** (Chiang et al., 2024)

18. 모델은 이메일이 스팸일 확률을 출력합니다. 이메일이 스팸으로 표시되는 임계값을 설정합니다. 예를 들어, 임계값을 50%로 설정하면, 분류 모델이 이메일이 스팸일 확률이 최소 50%라고 판단하면 스팸으로 표시됩니다.

19. **Arg max 함수**를 수행합니다.

20. 언더플로우 문제는 숫자가 너무 작아 주어진 형식에서 표현될 수 없을 때 발생하며, 0으로 반올림됩니다.

21. 구체적으로, 작성 당시 OpenAI API는 상위 20개의 가장 가능성이 높은 토큰에 대한 **로그 확률**(logprobs)만 보여줍니다. 이전에는 사용자 제공 텍스트의 임의 로그 확률을 얻을 수 있도록 했지만, **2023년 9월에 중단**했습니다. Anthropic은 모델의 로그 확률을 공개하지 않습니다.

22. 유료 모델 API는 종종 출력 토큰 수당 요금을 청구합니다.

23. **테스트 타임 컴퓨트**(Test time compute)는 오늘날 더 일반적으로 사용되는 용어입니다. 테스트 실행에 필요한 컴퓨팅 양으로 해석될 수 있어 혼란스럽다고 생각합니다.

24. 깨지기 쉬운 모델에 대해 가장 적합한 작업은 그것을 다른 것으로 교체하는 것입니다.

25. 동일한 입력에 대해 여러 출력을 생성하는 비용을 줄이기 위해 할 수 있는 일이 몇 가지 있습니다. 예를 들어, 입력이 한 번 처리된 다음 모든 출력에 재사용될 수 있습니다.

26. 일부 미세 조정 서비스는 이를 자동으로 처리합니다. **OpenAI의 미세 조정 서비스**는 훈련 중 분류기 헤드를 추가할 수 있도록 했지만, 작성 시점에는 이 기능이 비활성화되었습니다.

27. 인터넷 밈에서 말하는 것처럼, **"확률은 낮지만, 절대 0은 아니다."**

28. 2023년 12월, 제가 조언하는 AI 회사의 고객 지원 요청을 3개월 동안 검토한 결과, 질문의 5분의 1이 AI 모델의 일관성 문제를 처리하는 방법에 관한 것이었습니다. Drew Houston(Dropbox의 CEO)와 Harrison Chase(Langchain의 CEO)와 함께한 2023년 7월 패널에서 우리는 모두 환각(hallucination)이 대부분의 AI 엔터프라이즈 사용 사례에서 가장 큰 장애물이라고 동의했습니다.